In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.base import BaseEstimator
import pandas as pd
import numpy as np
from surprise import Dataset, SVD
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import numpy as np
import string
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.base import BaseEstimator
from tqdm import tqdm
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from sklearn.preprocessing import MinMaxScaler
import random

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## Преобработка

In [3]:
df_info = pd.read_csv('Course_info.csv')
df_coment = pd.read_csv('Comments.csv')

KeyboardInterrupt: 

In [17]:
#формат даты был неудобный и таблица была огромной с учетом того что данные с 2011 до 2022 года, сократила их до 2017-2022
df_coment['date'] = df_coment['date'].apply(lambda x: datetime.fromisoformat(x).date())
df_info['published_time'] = df_info['published_time'].apply(lambda x: datetime.fromisoformat(x).date())

df_coment['date'] = pd.to_datetime(df_coment['date'], errors='coerce')
df_info['published_time'] = pd.to_datetime(df_info['published_time'], errors='coerce')

# filtered_df_coment = df_coment[(df_coment['date'].dt.year >= 2021) & (df_coment['date'].dt.year <= 2022)]
# filtered_df_info = df_info[(df_info['published_time'].dt.year >= 2021) & (df_info['published_time'].dt.year <= 2022)]


In [18]:
filtered_df_coment = df_coment[df_coment['date'].dt.year == 2022]
filtered_df_info = df_info[df_info['published_time'].dt.year == 2022]

filtered_df_info.reset_index().to_csv('info2017.csv', index=False)
filtered_df_coment.reset_index().to_csv('comment2017.csv', index=False)

---

In [2]:
df_info = pd.read_csv('info2017.csv')
df_comment = pd.read_csv('comment2017.csv')

In [3]:
df_commen = df_comment.copy()

---
## Comments data - колаработирвня фильтрация

In [6]:
df_comment.display_name.nunique()

390724

In [5]:
#кто сколько комментов оставил
a = df_comment.groupby('display_name').agg({
    'id': 'count'
}).sort_values(by='id').reset_index()

In [23]:
a.describe()

,id
count,390724.000000
mean,3.894120
std,39.212408
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,9748.000000


In [6]:
# список пользователей с количеством комментариев > 2000
users_to_remove = a[a['id'] > 2200]['display_name']
df_commen = df_commen[~df_commen['display_name'].isin(users_to_remove)]

In [7]:
sample_users = df_commen['id'].drop_duplicates().sample(5000, random_state=42)
df_commen = df_commen[df_commen['id'].isin(sample_users)]

top_courses = df_commen['course_id'].value_counts().head(2500).index
df_commen = df_commen[df_commen['course_id'].isin(top_courses)]

# Проверим результат
print(f"Уникальных курсов: {df_commen['course_id'].nunique()}")
print(f"Уникальных пользователей: {df_commen['id'].nunique()}")
print(f"Размер финального датафрейма: {df_commen.shape}")

Уникальных курсов: 2500
Уникальных пользователей: 3633
Размер финального датафрейма: (3633, 7)


In [164]:
df_commen['userId'] = df_commen['display_name'].astype('category').cat.codes +2

# Шаг 2: Создать финальную таблицу
df_result = df_commen[['userId', 'course_id', 'rate', 'date', 'comment']].copy()

# Опционально: сортировка
df_ratings = df_result.sort_values(by=['userId']).reset_index(drop=True)
df_ratings.drop_duplicates(inplace=True, 
                           subset=['userId', 'course_id']
                           )

In [165]:
df_ratings['rate'] = df_ratings['rate'].round().astype(int)
df_ratings.rate.value_counts()

rate
5    2344
4     987
3     117
2      83
1      62
Name: count, dtype: int64

In [166]:
def train_test_split(X, ratio=0.2, user_col='userId', item_col='course_id',
                     rating_col='rate', time_col='date'):
    # сортируем оценки по времени
    X.sort_values(by=[time_col], inplace=True)
    # список всех юзеров
    userIds = X[user_col].unique()
    X_train_data = []
    X_test_data = []
    y_train = []
    y_test = []
    for userId in tqdm_notebook(userIds):
        curUser = X[X[user_col] == userId]
        # определяем позицию, по которой делим выборку и размещаем данные по массивам
        idx = int(curUser.shape[0] * (1 - ratio))
        X_train_data.append(curUser[[user_col, item_col]].iloc[:idx, :].values)
        X_test_data.append(curUser[[user_col, item_col]].iloc[idx:, :].values)
        y_train.append(curUser[rating_col].values[:idx])
        y_test.append(curUser[rating_col].values[idx:])
    # cтекуем данные по каждому пользователю в общие массивы
    X_train = pd.DataFrame(np.vstack(X_train_data), columns=[user_col, item_col])
    X_test = pd.DataFrame(np.vstack(X_test_data), columns=[user_col, item_col])
    y_train = np.hstack(y_train)
    y_test = np.hstack(y_test)
    return X_train, X_test, y_train, y_test

In [167]:
X_train, X_test, y_train, y_test = train_test_split(df_ratings)

C:\Users\user\AppData\Local\Temp\ipykernel_6184\1727997363.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for userId in tqdm_notebook(userIds):


  0%|          | 0/2905 [00:00<?, ?it/s]

In [168]:
X_train.shape, len(y_train), X_test.shape, len(y_test)

((648, 2), 648, (2945, 2), 2945)

---
## User Based

In [169]:
class UserBased(BaseEstimator):
    def fit(self, X, y, user_col='userId', item_col='course_id'):
        X = X.copy()
        # сохраним текущих пользователей и имеющиеся предметы
        self.users = X[user_col].unique()
        self.items = X[item_col].unique()
        
        X['y'] = y
        # рассчитаем среднее значение рейтинга для пользователя и предмета
        self.mean_y_user = X.groupby(user_col)['y'].mean()
        self.mean_y_item = X.groupby(item_col)['y'].mean()
        
        # вычитаем среднюю оценку пользователя
        X['y'] -= X[user_col].apply(lambda x: self.mean_y_user[x])
        
        # создаём векторы для каждого пользователя из просмотренных фильмов
        # для неизвестных фильмов ставим оценку 0
        self.user_ratings = pd.pivot_table(X, values='y', index=user_col,
                                           columns=item_col, fill_value=0)
        
        # считаем попарную схожесть между юзерами
        self.user_sim = cosine_similarity(self.user_ratings)
        # также сделаем словарь - {значение user_col: index в user_ratings}
        self.user_pos = dict()
        for user in self.users:
            self.user_pos[user] = np.argwhere(self.user_ratings.index.values == user)[0][0]
        return self
    
    def predict_rating(self, pr_user, pr_item):
        # если в обучающей выборке нет такого предмета
        # или пользователя, то вернём 0
        if not pr_item in self.items or not pr_user in self.users:
            return 3
    
        sim_vector = self.user_sim[self.user_pos[pr_user]]
        item_ratings = self.user_ratings.loc[:, pr_item]
        
        numerator = sim_vector.dot(item_ratings)
        denominator = sim_vector.sum() - 1
        
        if denominator == 0:
            pred = self.mean_y_user[pr_user]
        else:
            pred = self.mean_y_user[pr_user] + numerator / denominator
        
        # Ограничим предсказание значениями от 1 до 5
        return np.clip(pred, 1.0, 5.0)

    
    def predict(self, X, user_col='userId', item_col='course_id'):
        y = X[[user_col, item_col]].apply(lambda row: self.predict_rating(row[0], row[1]), axis=1)
        return y

In [170]:
%%time
print('start fitting...')
ub = UserBased().fit(X_train, y_train)
print('start predicting...')
y_pred = ub.predict(X_test)
# print('rmse = {}'.format(rmse(y_test,y_pred )))

start fitting...
start predicting...


C:\Users\user\AppData\Local\Temp\ipykernel_6184\994343726.py:51: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = X[[user_col, item_col]].apply(lambda row: self.predict_rating(row[0], row[1]), axis=1)


CPU times: total: 891 ms
Wall time: 397 ms


In [171]:
y_pred = y_pred.fillna(3)
y_pred.isnull().sum()

0

In [413]:
def evaluate_regression(y_true, y_pred):
    print("RMSE:", np.sqrt(mean_squared_error(y_true, y_pred)))
    print("MAE:", mean_absolute_error(y_true, y_pred))
    print("R2:", r2_score(y_true, y_pred))
    print("MAPE:", mean_absolute_percentage_error(y_true, y_pred))

# evaluate_regression(y_test, y_pred)

In [174]:
y_pred2 = ub.predict(X_train)
y_pred2 = y_pred2.fillna(3)
y_pred2 = y_pred2.round()
evaluate_regression(y_train, y_pred2)

C:\Users\user\AppData\Local\Temp\ipykernel_6184\994343726.py:51: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = X[[user_col, item_col]].apply(lambda row: self.predict_rating(row[0], row[1]), axis=1)


RMSE: 1.592807909647939
MAE: 0.8024691358024691
R2: -3.0693691078277077
MAPE: 0.20612139917695474


---
## Item Based

In [175]:
class ItemBased(BaseEstimator):
    def fit(self, X, y, user_col='userId', item_col='course_id'):
        X = X.copy()
        # сохраним текущих пользователей и имеющиеся предметы
        self.users = X[user_col].unique()
        self.items = X[item_col].unique()
        
        X['y'] = y
        # рассчитаем среднее значение рейтинга для пользователя и предмета
        self.mean_y_user = X.groupby(user_col)['y'].mean()
        self.mean_y_item = X.groupby(item_col)['y'].mean()
        
        # вычитаем среднюю оценку предмета
        X['y'] -= X[item_col].apply(lambda x: self.mean_y_item[x])
        
        # создаём векторы для каждого фильма с оценками пользователя
        # если пользователь не поставил оценку, то ставим 0
        self.item_ratings = pd.pivot_table(X, values='y', index=item_col,
                                           columns=user_col, fill_value=0)
        
        # считаем попарную схожесть между фильмами
        self.item_sim = cosine_similarity(self.item_ratings)
        
        # также сделаем словарь {значение item_col: index в item_ratings}
        self.item_pos = dict()
        for item in self.items:
            self.item_pos[item] = np.argwhere(self.item_ratings.index.values == item)[0][0]
        return self
    
    def predict_rating(self, pr_user, pr_item):
        # если в обучающей выборке нет такого предмета
        # или пользователя, то вернём 0
        if not pr_item in self.items or not pr_user in self.users:
            return 0
        
        # считаем числитель и знаменатель дроби из формулы предсказания
        numerator = self.item_sim[self.item_pos[pr_item]].dot(
                        self.item_ratings.loc[:, pr_user])   
        # вычитаем 1, так как схожесть предмета с самим собой равна 1,
        # но модель не должна это учитывать
        denominator = self.item_sim[self.item_pos[pr_item]].sum() - 1
        
        return self.mean_y_item[pr_item] + numerator / denominator

    def predict(self, X, user_col='userId', item_col='course_id'):
        y = X[[user_col, item_col]].apply(lambda row: self.predict_rating(row[0], row[1]), axis=1)
        return y

In [176]:
%%time
print('start fitting...')
ib = ItemBased().fit(X_train, y_train)
print('start predicting...')
y_pred = ib.predict(X_test)

start fitting...
start predicting...


C:\Users\user\AppData\Local\Temp\ipykernel_6184\2454844154.py:46: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = X[[user_col, item_col]].apply(lambda row: self.predict_rating(row[0], row[1]), axis=1)


CPU times: total: 938 ms
Wall time: 402 ms


In [177]:
evaluate_regression(y_test, y_pred)

RMSE: 4.5436546865574305
MAE: 4.426521950036381
R2: -29.89298694617109
MAPE: 0.9790395343196701


---
## Course data - контентная фильтрация

In [178]:
df_inf = df_info.copy()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [180]:
df_inf.columns

Index(['index', 'id', 'title', 'is_paid', 'price', 'headline',
       'num_subscribers', 'avg_rating', 'num_reviews', 'num_comments',
       'num_lectures', 'content_length_min', 'published_time',
       'last_update_date', 'category', 'subcategory', 'topic', 'language',
       'course_url', 'instructor_name', 'instructor_url'],
      dtype='object')

In [181]:
cols_to_combine = ['language', 'topic', 'instructor_name', 'category', 'subcategory', 'headline', 'title' ] 
# Обработка: приводим к нижнему регистру и заполняем пропуски
df_inf[cols_to_combine] = df_inf[cols_to_combine].fillna('').apply(lambda col: col.str.lower())
df_inf['description'] = df_inf[cols_to_combine].apply(lambda row: ' '.join(filter(None, row)), axis=1)


In [182]:
df_ratings

,userId,course_id,rate,date,comment
437,209,3426446,5,2022-01-01,Es un curso bastante entretenido y muy bien ex...
2301,1759,2359992,5,2022-01-01,Cause it's good
1371,1005,1503540,5,2022-01-01,Ótimo curso! O Prof. Willian é bem didático. V...
2586,1997,821986,5,2022-01-01,useful concepts are covered
2767,2132,1341268,4,2022-01-01,Boa apresentação.
...,...,...,...,...,...
1201,858,1178124,5,2022-10-22,[Español] El curso esta muy bien muy recomenda...
2853,2207,1978132,5,2022-10-22,Good elaborations of topics & easy to understa...
1390,1021,2013892,4,2022-10-22,I am basically from banking domain with core b...
3562,2836,2405042,4,2022-10-22,細かい部分まで丁寧に教えてくださっています。


In [184]:
sample = df_inf[['id', 'title', 'description']]
# определите model_index и сделайте его строкой
sample = sample.reset_index().rename(columns = {'index': 'model_index'})
sample['model_index'] = sample['model_index'].astype(str)
sample.head(5)

,model_index,id,title,description
0,0,84482.0,ovation public speaking - speaking methods mas...,english public speaking ovation public speakin...
1,1,84660.0,method lesson 4: speaking with modulation,english public speaking ovation public speakin...
2,2,84664.0,method lesson 5: speaking with enthusiasm,english public speaking ovation public speakin...
3,3,84626.0,method lesson 3: speaking with power,english public speaking ovation public speakin...
4,4,85752.0,content lesson 2: subject development,english presentation skills ovation public spe...


In [185]:
movies_inv_mapper = dict(zip(sample['title'].str.lower(), sample['model_index'].astype(int)))
movies_inv_mapper

{'ovation public speaking - speaking methods master class': 0,
 'method lesson 4: speaking with modulation': 1,
 'method lesson 5: speaking with enthusiasm': 2,
 'method lesson 3: speaking with power': 3,
 'content lesson 2: subject development': 4,
 'content lesson 1:  writing introductions': 5,
 'method lesson 6: speaking with gestures': 6,
 'step-by-step mind mastery course': 7,
 'game banao - unity 3d/2d game development in اردو / हिन्दी': 8,
 'learn the art of horsehair hitching': 9,
 'become a master wordsmith: learn 250+ advance level words': 10,
 'fundamentals of engineering (fe) exam review': 11,
 'next level js - modernes javascript mit es6 und neuer': 12,
 'redirecting negative behavior': 13,
 'financial literacy - what i wish i learned in school': 14,
 'program manager introduction course': 15,
 'business analysis: eliciting nonfunctional requirements': 16,
 'business analysis: four core models for scoping requirements': 17,
 'stress management in 3d: reduce stress and prev

In [186]:
punctuation = set(string.punctuation)
def word_tokenize_clean(doc: str, stop_words: list):
    '''
    Tokenize, clean and lemmatize English text
    '''
    # Приводим в нижний регистр и удаляем не-буквы
    doc = re.sub(r'[^a-zA-Z\s]', '', doc.lower())
    
    # Токенизация
    tokens = word_tokenize(doc)
    
    # Фильтрация: убираем стоп-слова, пунктуацию, лемматизируем
    tokens = [
        lemmatizer.lemmatize(word)
        for word in tokens
        if word.isalpha() and word not in stop_words and word not in punctuation
    ]
    
    return tokens

In [187]:
tags_corpus = [re.sub(r'[^a-zA-Z\s]', ' ', x) for x in df_inf['description']] 
tags_doc = [TaggedDocument(words=word_tokenize_clean(doc, stop_words), tags=[str(i)])
            for i, doc in enumerate(tags_corpus)]

tags_doc[0]


TaggedDocument(words=['english', 'public', 'speaking', 'ovation', 'public', 'speaking', 'business', 'communication', 'public', 'speaking', 'crucial', 'skill', 'order', 'make', 'advancement', 'education', 'career', 'personal', 'life', 'ovation', 'public', 'speaking', 'speaking', 'method', 'master', 'class'], tags=['0'])

In [188]:
model = Doc2Vec(vector_size = 50,
                alpha = .02, 
                min_alpha = .0003,
                min_count = 5,
                dm = 0)

In [189]:
model.build_vocab(tags_doc)
model.train(tags_doc,
            total_examples = model.corpus_count,
            epochs = 25)

In [190]:
sample['vector'] = [model.dv[str(i)] for i in range(len(sample))]

In [191]:
sample.title.unique().tolist()

['ovation public speaking - speaking methods master class',
 'method lesson 4: speaking with modulation',
 'method lesson 5: speaking with enthusiasm',
 'method lesson 3: speaking with power',
 'content lesson 2: subject development',
 'content lesson 1:  writing introductions',
 'method lesson 6: speaking with gestures',
 'step-by-step mind mastery course',
 'game banao - unity 3d/2d game development in اردو / हिन्दी',
 'learn the art of horsehair hitching',
 'become a master wordsmith: learn 250+ advance level words',
 'fundamentals of engineering (fe) exam review',
 'next level js - modernes javascript mit es6 und neuer',
 'redirecting negative behavior',
 'financial literacy - what i wish i learned in school',
 'program manager introduction course',
 'business analysis: eliciting nonfunctional requirements',
 'business analysis: four core models for scoping requirements',
 'stress management in 3d: reduce stress and prevent symptoms.',
 'grab life by the obstacles - be your own lif

In [192]:
def recommend(anime_name: str, top_n: int = 20):
    anime_index = sample[sample['title'] == anime_name].index[0]
    anime_vectors = model.dv.vectors
    movie_embeddings = anime_vectors[anime_index]

    all_vectors = sample['vector'].tolist()
    similarities = cosine_similarity([movie_embeddings], all_vectors)[0]

    similar_indices = similarities.argsort()[::-1][1:top_n+1]
    print(f"\n🔎 Рекомендации для: {anime_name}\n")
    recommendations = []
    for i in similar_indices:
        title = sample.iloc[i]['title']
        score = similarities[i]
        print(f"{title} (score: {score:.4f})")
        recommendations.append(title)

In [193]:
recommendations = recommend("data science on sustainable development goals (sdgs)")
recommendations


🔎 Рекомендации для: data science on sustainable development goals (sdgs)

how to start a career in data science? (score: 0.9318)
how to start career in data science (advice from 6 experts) (score: 0.9268)
how to get a data science job without experience (score: 0.9178)
applied ml: the big picture (score: 0.9120)
effective written guidance (score: 0.9071)
starting a career in data science 101 (score: 0.9062)
industry 4.0, it's my time! (score: 0.9050)
data analysis with polars (score: 0.9032)
kaggle - get the best data science, machine learning profile (score: 0.8935)
be aware of data science (score: 0.8931)
desmos fundamentals (score: 0.8928)
ocean data in canada (score: 0.8890)
knime a platform for machine learning and data science (score: 0.8846)
deep learning (score: 0.8841)
fundamentals of machine learning (score: 0.8831)
the new future with ai basic understanding & ethics (score: 0.8801)
real quick start to learning data science (score: 0.8784)
python data science with the tclab 

---
## Гибридная каскадная модель

In [196]:
df_ratings =  df_ratings[['userId', 'course_id', 'rate']] # Таблица с оценками (user_id, course_id, rate)
df_inf = df_inf[['id', 'title', 'description']]      # Метаданные курсов (course_id, title, description, ...) 
df_comments = df_commen[['userId', 'course_id', 'comment']] # Комментарии (user_id, course_id, text)

In [199]:
# Ваши обученные модели (у вас уже есть)
cf_model = UserBased()  # Ваш CF-класс
doc2vec_model = model   # Ваш Doc2Vec/Doc2Vec-like модель

In [200]:
def get_cf_recommendations(user_id, cf_model, n=100):
    # Все уникальные курсы
    all_courses = df_inf['id'].unique()  
    # Предсказать оценки для всех курсов
    predictions = [cf_model.predict_rating(user_id, course) for course in all_courses]
    # Топ-N курсов с наивысшими оценками
    top_n = np.argsort(predictions)[-n:][::-1]  
    return all_courses[top_n]

In [201]:
def get_content_scores(user_id, cf_candidates):
    # 1. Вектор предпочтений пользователя (усреднение векторов пройденных курсов)
    user_courses = df_ratings[df_ratings['userId'] == user_id]['course_id']
    user_courses_vectors = [model.dv[str(course)] for course in user_courses]
    user_profile = np.mean(user_courses_vectors, axis=0) if user_courses_vectors else np.zeros(50)
    
    # 2. Схожесть с кандидатами из CF
    candidates_vectors = [model.dv[str(course)] for course in cf_candidates]
    content_scores = cosine_similarity([user_profile], candidates_vectors)[0]
    return content_scores

In [202]:
def get_comment_bonus(user_id, course_id):
    # 1. Эмбеддинг комментариев пользователя
    user_comments = df_comments[df_comments['userId'] == user_id]['comment']
    user_comment_embedding = np.mean([model.infer_vector(comment.split()) for comment in user_comments], axis=0)
    
    # 2. Эмбеддинг отзывов на курс
    course_comments = df_comments[df_comments['course_id'] == course_id]['comment']
    course_comment_embedding = np.mean([model.infer_vector(comment.split()) for comment in course_comments], axis=0)
    
    # 3. Косинусная схожесть
    return cosine_similarity([user_comment_embedding], [course_comment_embedding])[0][0]

In [204]:
def hybrid_recommend(user_id, cf_model, top_n_cf=100, top_k_final=10):
    # 1. CF: Первичный отбор
    cf_candidates = get_cf_recommendations(user_id, cf_model, top_n_cf)
    
    # 2. Content-Based: Ранжирование
    content_scores = get_content_scores(user_id, cf_candidates)
    
    # 3. Комментарии: Коррекция
    comment_bonuses = [get_comment_bonus(user_id, course) for course in cf_candidates]
    
    # 4. Итоговый рейтинг (взвешивание)
    final_scores = 0.6 * content_scores + 0.4 * np.array(comment_bonuses)  # Веса можно настроить
    
    # 5. Топ-K курсов
    top_k_indices = np.argsort(final_scores)[-top_k_final:][::-1]
    return cf_candidates[top_k_indices], final_scores[top_k_indices]

In [208]:
# 1. Подготовка данных (пример)
# Предположим, у вас есть такие данные:
user_id = 209  # ID пользователя, для которого делаем рекомендации
ub = UserBased()  # Ваша обученная CF-модель
model = ...  # Ваша обученная Doc2Vec модель

# 2. Получаем рекомендации
try:
    recommended_courses, scores = hybrid_recommend(user_id, ub)
    
    # 3. Красивый вывод результатов
    print("\n🎓 Рекомендации для пользователя ID:", user_id)
    print("----------------------------------------")
    
    # Создаем DataFrame для удобного отображения
    results = pd.DataFrame({
        'course_id': recommended_courses,
        'score': scores,
        'title': df_inf.loc[df_inf['course_id'].isin(recommended_courses), 'title'].values,
        'rating': df_inf.loc[df_inf['course_id'].isin(recommended_courses), 'avg_rating'].values
    }).sort_values('score', ascending=False)
    
    # Выводим топ-5 курсов для примера
    print(results.head(5).to_string(index=False))
    
    # Дополнительная аналитика
    print("\n📊 Статистика рекомендаций:")
    print(f"- Средний рейтинг рекомендованных курсов: {results['rating'].mean():.2f}")
    print(f"- Максимальный score: {results['score'].max():.2f}")
    print(f"- Минимальный score: {results['score'].min():.2f}")

except Exception as e:
    print(f"❌ Ошибка: {str(e)}")
    print("Проверьте, что:")
    print("1. Модели (CF и Doc2Vec) корректно обучены")
    print("2. User ID существует в ваших данных")
    print("3. Все необходимые DataFrame (df_inf, df_ratings) загружены")

❌ Ошибка: 'UserBased' object has no attribute 'items'
Проверьте, что:
1. Модели (CF и Doc2Vec) корректно обучены
2. User ID существует в ваших данных
3. Все необходимые DataFrame (df_inf, df_ratings) загружены


In [ ]:


# Инициализация NLP
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# 1. Загрузка данных (пример структуры)
df_ratings = pd.DataFrame({
    'user_id': [1, 1, 2, 2, 3, 3],
    'course_id': [101, 102, 101, 103, 102, 103],
    'rate': [5, 4, 3, 5, 2, 4],
    'date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06']
})

df_courses = pd.DataFrame({
    'course_id': [101, 102, 103],
    'title': ['Python Basics', 'Data Science', 'Machine Learning'],
    'description': ['Learn Python programming', 'DS with Python', 'ML fundamentals'],
    'category': ['Programming', 'Data', 'Data'],
    'avg_rating': [4.5, 4.2, 4.8]
})

df_comments = pd.DataFrame({
    'user_id': [1, 2, 3],
    'course_id': [101, 102, 103],
    'text': ['Great course!', 'Too difficult', 'Perfect for beginners']
})

In [209]:
# 2. Подготовка NLP модели
def tokenize(text):
    tokens = word_tokenize(re.sub(r'[^a-zA-Z\s]', '', text.lower()))
    return [word for word in tokens if word not in stop_words]

# Создаем TaggedDocument для Doc2Vec
documents = [TaggedDocument(tokenize(row['description']), [str(row['course_id'])]) 
             for _, row in df_courses.iterrows()]

# Обучаем модель
model = Doc2Vec(vector_size=50, min_count=1, epochs=20)
model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

# 3. Реализация User-Based CF
class UserBasedCF(BaseEstimator):
    def __init__(self):
        self.user_sim = None
        self.user_ratings = None
        self.users = None
        self.items = None
        
    def fit(self, X, y, user_col='user_id', item_col='course_id'):
        self.users = X[user_col].unique()
        self.items = X[item_col].unique()
        
        X = X.copy()
        X['y'] = y
        self.mean_y = X['y'].mean()
        self.mean_y_user = X.groupby(user_col)['y'].mean()
        self.mean_y_item = X.groupby(item_col)['y'].mean()
        
        X['y'] -= X[user_col].apply(lambda x: self.mean_y_user[x])
        
        self.user_ratings = pd.pivot_table(X, values='y', index=user_col,
                                        columns=item_col, fill_value=0)
        
        self.user_sim = cosine_similarity(self.user_ratings)
        self.user_pos = {user: idx for idx, user in enumerate(self.user_ratings.index)}
        
    def predict(self, user_id, item_id):
        if user_id not in self.user_pos or item_id not in self.items:
            return self.mean_y
        
        user_idx = self.user_pos[user_id]
        sim_scores = self.user_sim[user_idx]
        item_ratings = self.user_ratings.get(item_id, pd.Series(0, index=self.user_ratings.index))
        
        numerator = np.dot(sim_scores, item_ratings)
        denominator = np.sum(np.abs(sim_scores)) - 1  # Исключаем самого пользователя
        
        if denominator == 0:
            return self.mean_y_user.get(user_id, self.mean_y)
        
        pred = self.mean_y_user.get(user_id, self.mean_y) + numerator / denominator
        return np.clip(pred, 1, 5)

# 4. Обучение CF модели
cf_model = UserBasedCF()
cf_model.fit(df_ratings, df_ratings['rate'])

# 5. Гибридная рекомендация
def hybrid_recommend(user_id, cf_model, n_candidates=50, top_k=5):
    try:
        # CF: Получаем кандидатов
        all_items = df_courses['course_id'].unique()
        cf_scores = [cf_model.predict(user_id, item) for item in all_items]
        cf_candidates = np.array(all_items)[np.argsort(cf_scores)[-n_candidates:][::-1]]
        
        # Content: Вектор пользователя (по истории)
        user_history = df_ratings[df_ratings['user_id'] == user_id]['course_id']
        if len(user_history) == 0:
            return df_courses.sort_values('avg_rating', ascending=False)['course_id'].head(top_k).tolist()
        
        user_vector = np.mean([model.dv[str(item)] for item in user_history], axis=0)
        
        # Content: Схожесть с кандидатами
        content_scores = cosine_similarity([user_vector], 
                                         [model.dv[str(item)] for item in cf_candidates])[0]
        
        # Комментарии: Анализ тональности (упрощенный)
        comment_bonus = np.ones(len(cf_candidates))
        if user_id in df_comments['user_id'].values:
            user_comments = ' '.join(df_comments[df_comments['user_id'] == user_id]['text'])
            user_tokens = set(tokenize(user_comments))
            
            for i, item in enumerate(cf_candidates):
                item_comments = ' '.join(df_comments[df_comments['course_id'] == item]['text'])
                item_tokens = set(tokenize(item_comments))
                comment_bonus[i] = len(user_tokens & item_tokens) / len(user_tokens | item_tokens) if len(user_tokens | item_tokens) > 0 else 0
        
        # Итоговый рейтинг
        final_scores = 0.6 * content_scores + 0.4 * comment_bonus
        top_indices = np.argsort(final_scores)[-top_k:][::-1]
        
        return cf_candidates[top_indices], final_scores[top_indices]
    
    except Exception as e:
        print(f"Error: {str(e)}")
        return [], []

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [211]:

# 6. Пример использования
user_id = 1
courses, scores = hybrid_recommend(user_id, cf_model)
print(f"Рекомендации для пользователя {user_id}:")
for course, score in zip(courses, scores):
    title = df_courses[df_courses['course_id'] == course]['title'].values[0]
    print(f"- {title} (score: {score:.2f})")

Рекомендации для пользователя 1:
- Python Basics (score: 0.88)
- Data Science (score: 0.48)
- Machine Learning (score: 0.02)


In [206]:
df_ratings

,userId,course_id,rate
437,209,3426446,5
2301,1759,2359992,5
1371,1005,1503540,5
2586,1997,821986,5
2767,2132,1341268,4
...,...,...,...
1201,858,1178124,5
2853,2207,1978132,5
1390,1021,2013892,4
3562,2836,2405042,4


---
# OMG

In [142]:
df_info = pd.read_csv('info2017.csv')
df_comment = pd.read_csv('comment2017.csv')

In [144]:
df_info.shape, df_comment.shape

((36124, 21), (1578068, 7))

In [239]:
df_info_final.shape, df_comment_final.shape

((1005, 18), (3176, 5))

In [41]:
english_courses = df_info[df_info['language'].str.lower() == 'english']

df_comment['date'] = pd.to_datetime(df_comment['date'], errors='coerce')
df_comment_2022 = df_comment[df_comment['date'].dt.year == 2022]

comment_counts = df_comment_2022['display_name'].value_counts()
valid_users = comment_counts[(comment_counts >= 5) & (comment_counts <= 100)].index
df_comment_filtered = df_comment_2022[df_comment_2022['display_name'].isin(valid_users)]

# Удаляем неанглийские комментарии на основе ascii
df_comment_filtered = df_comment_filtered[df_comment_filtered['comment'].apply(lambda x: isinstance(x, str) and x.isascii())]

# Оставим курсы, которые есть в `english_courses`
df_comment_filtered = df_comment_filtered[df_comment_filtered['course_id'].isin(english_courses['id'])]


# Ограничим до нужного количества:
# top_courses = df_comment_filtered['course_id'].value_counts().head(1563).index
# top_users = df_comment_filtered['display_name'].value_counts().head(4922).index

# df_comment_filtered = df_comment_filtered[
#     df_comment_filtered['course_id'].isin(top_courses) & 
#     df_comment_filtered['display_name'].isin(top_users)
# ]

# Шаг 1: Найти топ-40 курсов по количеству комментариев
top_40_courses = (
    df_comment_filtered['course_id']
    .value_counts()
    .head(30)
    .index
)

# Шаг 2: Отфильтровать комментарии только по этим курсам
filtered_by_top_courses = df_comment_filtered[df_comment_filtered['course_id'].isin(top_40_courses)]

# Шаг 3: Найти топ-500 комментаторов по этим курсам
top_500_users = (
    filtered_by_top_courses['display_name']
    .value_counts()
    .head(700)
    .index
)

# Шаг 4: Финальный фильтр — только комментарии от топ-500 пользователей по топ-40 курсам
df_top_comments = filtered_by_top_courses[
    filtered_by_top_courses['display_name'].isin(top_500_users)
]

# Заменяем df_comment_filtered на финальный отфильтрованный набор
df_comment_filtered = df_top_comments.reset_index(drop=True)

# Пересоздаем df_info_filtered только для топ-40 курсов
df_info_filtered = english_courses[english_courses['id'].isin(df_comment_filtered['course_id'])].reset_index(drop=True)

# Обновляем индексы
df_comment_filtered.index += 1
df_info_filtered.index += 1

# Уникальные пользователи и курсы — только из отфильтрованного набора
user_mapping = {name: idx for idx, name in enumerate(df_comment_filtered['display_name'].unique(), start=1)}
course_mapping = {cid: idx for idx, cid in enumerate(df_comment_filtered['course_id'].unique(), start=1)}

# Применяем маппинг
df_comment_filtered['user_id'] = df_comment_filtered['display_name'].map(user_mapping)
df_comment_filtered['course_id'] = df_comment_filtered['course_id'].map(course_mapping)

df_info_filtered['course_id'] = df_info_filtered['id'].map(course_mapping)

# Финальные таблицы
df_comment_final = df_comment_filtered.drop(columns=['display_name', 'index', 'id'], errors='ignore')
df_info_final = df_info_filtered.drop(columns=['id', 'index', 'language', 'instructor_url'], errors='ignore')

# Проверим
print("Уникальных курсов:", df_info_final['course_id'].nunique())  # Должно быть 40
print("Уникальных пользователей:", df_comment_final['user_id'].nunique())  # Должно быть <= 500


# valid_course_ids = set(english_courses['id'])
# used_course_ids = set(df_comment_filtered['course_id'])

# assert used_course_ids.issubset(valid_course_ids), "Найдены курсы, которых нет в df_info"

# # Также можно сократить df_info до используемых курсов:
# df_info_filtered = english_courses[english_courses['id'].isin(df_comment_filtered['course_id'])]

# df_comment_filtered = df_comment_filtered.reset_index(drop=True)
# df_info_filtered = df_info_filtered.reset_index(drop=True)

# df_comment_filtered.index += 1
# df_info_filtered.index += 1

# # Уникальные пользователи и курсы
# user_mapping = {name: idx for idx, name in enumerate(df_comment_filtered['display_name'].unique(), start=1)}
# course_mapping = {cid: idx for idx, cid in enumerate(df_comment_filtered['course_id'].unique(), start=1)}

# df_comment_filtered['user_id'] = df_comment_filtered['display_name'].map(user_mapping)
# df_comment_filtered['course_id'] = df_comment_filtered['course_id'].map(course_mapping)

# df_info_filtered['course_id'] = df_info_filtered['id'].map(course_mapping)

# df_comment_final = df_comment_filtered.drop(columns=['display_name', 'index', 'id'])
# df_info_final = df_info_filtered.drop(columns=['id', 'index', 'language','instructor_url'])

Уникальных курсов: 30
Уникальных пользователей: 700


In [35]:
df_comment_final.to_csv('comment2022_final.csv', index=False)
df_info_final.to_csv('info2022_final.csv', index=False)

In [42]:
# courses = pd.read_csv('info2022_final.csv')
# users = pd.read_csv('comment2022_final.csv')
courses = df_info_final.copy()
users = df_comment_final.copy()

In [37]:
courses.course_id.nunique()

35

In [18]:
users.user_id.nunique()

500

---
# От сюда!!!!

In [244]:
from sklearn.metrics import mean_squared_error, ndcg_score

# Ваши данные (пример)
df_ratings = users[['user_id', 'course_id', 'rate', 'date']] # user_id, course_id, rate, date
df_ratings.drop_duplicates(inplace=True) 
df_courses =  courses.copy()
df_comments = users[['user_id', 'course_id', 'comment']] # user_id, course_id, text

# Ваша функция split (адаптированная)
def train_test_split(X, ratio=0.40, user_col='user_id', item_col='course_id', 
                    rating_col='rate', time_col='date'):
    X.sort_values(by=[time_col], inplace=True)
    users = X[user_col].unique()
    
    X_train, X_test = [], []
    y_train, y_test = [], []
    
    for user in tqdm(users):
        user_data = X[X[user_col] == user]
        split_idx = int(len(user_data) * (1 - ratio))
        
        X_train.append(user_data[[user_col, item_col]].iloc[:split_idx])
        X_test.append(user_data[[user_col, item_col]].iloc[split_idx:])
        y_train.append(user_data[rating_col].iloc[:split_idx])
        y_test.append(user_data[rating_col].iloc[split_idx:])
    
    return (
        pd.concat(X_train), pd.concat(X_test),
        np.concatenate(y_train), np.concatenate(y_test)
    )

X_train, X_test, y_train, y_test = train_test_split(df_ratings)

C:\Users\user\AppData\Local\Temp\ipykernel_13132\40192615.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ratings.drop_duplicates(inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_13132\40192615.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.sort_values(by=[time_col], inplace=True)
100%|██████████| 700/700 [00:01<00:00, 635.98it/s]


In [245]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.base import BaseEstimator

class UserBasedCF(BaseEstimator):
    def __init__(self):
        self.user_sim = None
        self.user_ratings = None
        self.users = None
        self.items = None
        self.mean_y_user = None
        self.mean_y_item = None
        self.user_pos = None
        self.global_mean = None
    
    def fit(self, X, y, user_col='user_id', item_col='course_id'):
        X = X.copy()
        self.users = X[user_col].unique()
        self.items = X[item_col].unique()
        
        X['y'] = y
        self.global_mean = X['y'].mean()
        self.mean_y_user = X.groupby(user_col)['y'].mean()
        self.mean_y_item = X.groupby(item_col)['y'].mean()
        
        # Центрирование оценок
        X['y'] -= X[user_col].apply(lambda x: self.mean_y_user[x])
        
        # Создание user-item матрицы
        self.user_ratings = pd.pivot_table(
            X, 
            values='y', 
            index=user_col,
            columns=item_col, 
            fill_value=0
        )
        
        # Матрица схожести пользователей
        self.user_sim = cosine_similarity(self.user_ratings)
        
        # Позиции пользователей в матрице
        self.user_pos = {user: idx for idx, user in enumerate(self.user_ratings.index)}
        
        return self
    
    def predict_rating(self, user_id, item_id):
        """Предсказание оценки для одного user-item pair"""
        if item_id not in self.items or user_id not in self.users:
            return self.global_mean
        
        if user_id not in self.user_pos:
            return self.mean_y_item.get(item_id, self.global_mean)
        
        user_idx = self.user_pos[user_id]
        sim_scores = self.user_sim[user_idx]
        item_ratings = self.user_ratings.get(item_id, pd.Series(0, index=self.user_ratings.index))
        
        numerator = np.dot(sim_scores, item_ratings)
        denominator = np.sum(np.abs(sim_scores)) - 1  # Исключаем самого пользователя
        
        if denominator <= 0:
            return self.mean_y_user.get(user_id, self.global_mean)
        
        pred = self.mean_y_user.get(user_id, self.global_mean) + numerator / denominator
        return np.clip(pred, 1.0, 5.0)
    
    def predict(self, X, user_col='user_id', item_col='course_id'):
        """Предсказание для множества пар"""
        return X[[user_col, item_col]].apply(
            lambda row: self.predict_rating(row[user_col], row[item_col]), 
            axis=1
        )
    
cf_model = UserBasedCF()
cf_model.fit(X_train, y_train)
# 2. Предсказание на тестовых данных
y_pred_cf = cf_model.predict(X_test)

In [240]:
evaluate_regression(y_test, y_pred_cf)

RMSE: 0.6043043940317531
MAE: 0.29569327731092443
R2: -0.14053848090444077
MAPE: 0.09531897740263716


In [246]:
evaluate_regression(y_test, y_pred_cf)

RMSE: 0.6043043940317531
MAE: 0.29569327731092443
R2: -0.14053848090444077
MAPE: 0.09531897740263716


In [122]:
punctuation = set(string.punctuation)
def word_tokenize_clean(doc: str, stop_words: list):
    '''
    Tokenize, clean and lemmatize English text
    '''
    # Приводим в нижний регистр и удаляем не-буквы
    doc = re.sub(r'[^a-zA-Z\s]', '', doc.lower())
    tokens = word_tokenize(doc)
    tokens = [
        lemmatizer.lemmatize(word)
        for word in tokens
        if word.isalpha() and word not in stop_words and word not in punctuation
    ]
    return tokens

string_cols = df_courses.select_dtypes(include='object')
string_cols = string_cols.fillna('').apply(lambda col: col.str.lower())
df_courses['description'] = string_cols.apply(lambda row: ' '.join(filter(None, row)), axis=1)

tags_corpus = [re.sub(r'[^a-zA-Z\s]', ' ', x) for x in df_courses['description']] 
tags_doc = [TaggedDocument(words=word_tokenize_clean(doc, stop_words), tags=[str(i)])
            for i, doc in enumerate(tags_corpus)]

model =  Doc2Vec(vector_size = 50,
                alpha = .02, 
                min_alpha = .0003,
                min_count = 5,
                dm = 0)

model.build_vocab(tags_doc)
model.train(tags_doc,
            total_examples = model.corpus_count,
            epochs = 25)


In [123]:
class HybridCascadeRecommender:
    def __init__(self, cf_model, doc2vec_model, df_courses, X_train, n_candidates=100):
        self.cf_model = cf_model
        self.doc2vec_model = doc2vec_model
        self.df_courses = df_courses
        self.X_train = X_train
        self.n_candidates = n_candidates
        
        # Создаем маппинг course_id -> doc2vec tag
        self.course_id_to_tag = {cid: str(idx) for idx, cid in enumerate(df_courses['course_id'])}
        self.valid_course_ids = set(df_courses['course_id'])
        
    def _get_cf_candidates(self, user_id):
        """Этап 1: Отбор кандидатов с помощью CF"""
        valid_items = [cid for cid in self.df_courses['course_id'] if cid in self.valid_course_ids]
        scores = [self.cf_model.predict_rating(user_id, item) for item in valid_items]
        top_indices = np.argsort(scores)[-self.n_candidates:][::-1]
        return np.array(valid_items)[top_indices], np.array(scores)[top_indices]
    
    def _get_content_scores(self, user_id, candidates):
        """Этап 2: Переранжирование на основе контента"""
        user_history = self.X_train[self.X_train['user_id'] == user_id]['course_id']
        user_history = [cid for cid in user_history if cid in self.valid_course_ids]
        
        if len(user_history) == 0:
            return np.zeros(len(candidates))
        
        try:
            # Вектор предпочтений пользователя
            user_vector = np.mean([
                self.doc2vec_model.dv[self.course_id_to_tag[cid]] 
                for cid in user_history 
                if cid in self.course_id_to_tag
            ], axis=0)
            
            # Схожесть с кандидатами
            candidate_vectors = [
                self.doc2vec_model.dv[self.course_id_to_tag[cid]]
                for cid in candidates
                if cid in self.course_id_to_tag
            ]
            
            if len(candidate_vectors) == 0:
                return np.zeros(len(candidates))
                
            return cosine_similarity([user_vector], candidate_vectors)[0]
            
        except KeyError as e:
            print(f"Warning: {str(e)} - using fallback")
            return np.zeros(len(candidates))
    
    def recommend(self, user_id, top_k=10):
        """Каскадная рекомендация"""
        # Этап 1: CF отбор
        candidates, cf_scores = self._get_cf_candidates(user_id)
        
        # Этап 2: Контентное переранжирование
        content_scores = self._get_content_scores(user_id, candidates)
        
        # Комбинирование оценок
        combined_scores = 0.7 * cf_scores + 0.3 * content_scores
        
        # Возвращаем топ-K
        top_indices = np.argsort(combined_scores)[-top_k:][::-1]
        return candidates[top_indices], combined_scores[top_indices]

In [103]:
# Инициализация
hybrid_model = HybridCascadeRecommender(
    cf_model=cf_model,
    doc2vec_model=model,
    df_courses=courses,
    X_train=X_train,
    n_candidates=100
)

# Получение рекомендаций
user_id = 4 # Должен существовать в X_train
try:
    recommended, scores = hybrid_model.recommend(user_id)
    
    print(f"Рекомендации для пользователя {user_id}:")
    for course_id, score in zip(recommended, scores):
        title = df_courses[df_courses['course_id'] == course_id]['title'].values[0]
        print(f"- {title} (score: {score:.2f})")
        
except Exception as e:
    print(f"Ошибка: {str(e)}")
    print("Возможные причины:")
    print("1. Пользователь отсутствует в обучающих данных")
    print("2. Нет данных о курсах")

Рекомендации для пользователя 4:
- Google SEO MasterClass for Beginners (score: 3.80)
- Complete DevOps Ansible Automation Training (score: 3.80)
- PMBOK Guide 7th Edition Review 18 PDU's, Renew the PMP (score: 3.80)
- MERN Stack Course 2022 -  MongoDB, Express, React and NodeJS (score: 3.80)
- Mastering TypeScript - 2022 Edition (score: 3.80)
- The Ultimate Affiliate Marketing Course For Beginners 2022 (score: 3.80)
- Problem Solving: The Complete Guide (score: 3.80)
- Scrum Master PSM 1 - Scrum Master Certification - Agile 2022 (score: 3.80)
- Salesforce Administrator Certification - Pass in 2022 (Oct) (score: 3.80)
- Complete Blender Megacourse: Beginner to Expert (score: 3.80)


In [104]:
# Предсказание оценок на тестовых данных
y_pred_cf = cf_model.predict(X_test)

# Оценка
print("Оценка коллаборативной фильтрации:")
evaluate_regression(y_test, y_pred_cf)

Оценка коллаборативной фильтрации:
RMSE: 0.5965410738138304
MAE: 0.32103441894892665
R2: -0.1317346347299
MAPE: 0.09687977012912244


In [108]:
df_ratings.user_id.nunique()

700

---
# Surprise

In [706]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(df_ratings['rate'].min(), df_ratings['rate'].max()))

# Преобразуем DataFrame в surprise Dataset
data = Dataset.load_from_df(df_ratings[['user_id', 'course_id', 'rate']], reader)

# Разделение на train/test
trainset, testset = train_test_split(data, test_size=0.4, random_state=42)

# Обучение SVD
svd_model = SVD(n_factors=50, n_epochs=20, lr_all=0.005, reg_all=0.02)
svd_model.fit(trainset)

# Предсказания
y_pred = svd_model.test(testset)

In [110]:
from surprise import accuracy

# Метрики для SVD
print("Оценка SVD модели:")
y_true = [pred.r_ui for pred in y_pred]
y_pred = [pred.est for pred in y_pred]
evaluate_regression(y_true, y_pred)

def svd_recommend(user_id, n=10):
    test_user_items = df_ratings[df_ratings['user_id'] == user_id]['course_id']
    all_items = df_courses['course_id'].unique()
    unseen = [item for item in all_items if item not in test_user_items]
    
    predictions = []
    for item in unseen:
        pred = svd_model.predict(user_id, item)
        predictions.append((item, pred.est))
    
    predictions.sort(key=lambda x: x[1], reverse=True)
    return [x[0] for x in predictions[:n]], [x[1] for x in predictions[:n]]

# Пример для одного пользователя
test_user = X_test['user_id'].iloc[9]
svd_rec, svd_scores = svd_recommend(test_user)
true_items = X_test[X_test['user_id'] == test_user]['course_id'].values

Оценка SVD модели:
RMSE: 0.6075875665122697
MAE: 0.36654109357611325
R2: 0.019803249475817353
MAPE: 0.11996924182239019


In [111]:
from surprise.model_selection import GridSearchCV


In [112]:
# Задаем сетку параметров
param_grid = {'n_factors': [25, 50, 100, 200, 300],
              'n_epochs': [20, 30, 50, 70],
              'lr_all': [0.002, 0.005, 0.01],
              'reg_all': [0.02, 0.1, 0.2, 0.01]}

# Запускаем GridSearchCV
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5, joblib_verbose=1)
grid_search.fit(data)

# Лучшие параметры
print("Лучшие параметры по RMSE:", grid_search.best_params['rmse'])

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    3.8s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:   10.2s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:   23.0s


Лучшие параметры по RMSE: {'n_factors': 200, 'n_epochs': 30, 'lr_all': 0.01, 'reg_all': 0.2}


In [113]:
svd_model2 = SVD(n_factors=200, n_epochs=50, lr_all=0.01, reg_all=0.2)
svd_model2.fit(trainset)

y_pred2 = svd_model2.test(testset)

In [114]:
print("Оценка SVD модели:")
y_true = [pred.r_ui for pred in y_pred2]
y_pred = [pred.est for pred in y_pred2]
evaluate_regression(y_true, y_pred)

Оценка SVD модели:
RMSE: 0.6031375123101819
MAE: 0.3565035187925191
R2: 0.034108858174356715
MAPE: 0.1178052486584217


In [66]:
from surprise.model_selection import cross_validate

results1 = cross_validate(svd_model2, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.6435  0.5142  0.6118  0.5010  0.4453  0.5431  0.0734  
MAE (testset)     0.3555  0.3436  0.3797  0.3125  0.3212  0.3425  0.0241  
Fit time          0.07    0.06    0.06    0.05    0.06    0.06    0.01    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


In [67]:
results1 = cross_validate(svd_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.5126  0.4807  0.6316  0.6075  0.5456  0.5556  0.0566  
MAE (testset)     0.3333  0.3272  0.3711  0.3756  0.3513  0.3517  0.0194  
Fit time          0.01    0.02    0.01    0.01    0.01    0.01    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


In [115]:
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline

sim_options = {
    'name': 'cosine',
    'user_based': True  # False → item-based
}
algo = KNNBasic(user_based=True)
algo.fit(trainset)
pred_basic = algo.test(testset)

print(""" ---------------------------
      Оценка KNNBasic модели:""")
accuracy.rmse(pred_basic)
accuracy.mae(pred_basic)



means = KNNWithMeans(user_based=True)
means.fit(trainset)
pred_means = means.test(testset)

print(""" ---------------------------
      Оценка KNNWithMeans модели:""")
accuracy.rmse(pred_means)
accuracy.mae(pred_means)



z = KNNWithZScore(user_based=True)
z.fit(trainset)
pred_z = z.test(testset)

print(""" ---------------------------
      Оценка KNNWithZScore модели:""")
accuracy.rmse(pred_z)
accuracy.mae(pred_z)



base = KNNBaseline(user_based=True)
base.fit(trainset)
pred_base = base.test(testset)

print(""" ---------------------------
      Оценка KNNBaseline модели:""")
accuracy.rmse(pred_base)
accuracy.mae(pred_base)

Computing the msd similarity matrix...
Done computing similarity matrix.
 ---------------------------
      Оценка KNNBasic модели:
RMSE: 0.6265
MAE:  0.3673
Computing the msd similarity matrix...
Done computing similarity matrix.
 ---------------------------
      Оценка KNNWithMeans модели:
RMSE: 0.6228
MAE:  0.3463
Computing the msd similarity matrix...
Done computing similarity matrix.
 ---------------------------
      Оценка KNNWithZScore модели:
RMSE: 0.6216
MAE:  0.3456
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
 ---------------------------
      Оценка KNNBaseline модели:
RMSE: 0.6183
MAE:  0.3646


0.36457201288060237

In [276]:
param_grid = {
    'k': [3, 10, 20, 40, 60],
    'sim_options': {
        'name': ['cosine', 'pearson', 'msd'],
        'user_based': [True]
    }
}

gs = GridSearchCV(KNNBasic , param_grid, measures=['rmse'], cv=3, joblib_verbose=1)
gs.fit(data)

print("Лучшие параметры:", gs.best_params['rmse'])

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pea

In [272]:
param_grid = {
    'k': [3, 10, 20, 40, 60],
    'sim_options': {
        'name': ['pearson_baseline'],
        'user_based': [True]
    },
    'bsl_options': {
        'method': ['als'],
        'n_epochs': [5, 10, 15, 20],
        'reg_u': [10, 12, 15, 20],
        'reg_i': [5, 10]
    }
}
gs = GridSearchCV(KNNBaseline, param_grid, measures=['rmse'], cv=3)
gs.fit(data)
print("Лучшие параметры:", gs.best_params['rmse'])

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

In [116]:
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline

# 1. KNNBasic
algo = KNNBasic(
    k=3,
    sim_options={
        'name': 'pearson',
        'user_based': True
    }
)
algo.fit(trainset)
pred_basic = algo.test(testset)

print(""" ---------------------------
      Оценка KNNBasic модели:""")
y_true = [pred.r_ui for pred in pred_basic]
y_pred = [pred.est for pred in pred_basic]
evaluate_regression(y_true, y_pred)

# 2. KNNWithMeans
means = KNNWithMeans(
    k=3,
    sim_options={
        'name': 'pearson',
        'user_based': True
    }
)
means.fit(trainset)
pred_means = means.test(testset)

print(""" ---------------------------
      Оценка KNNWithMeans модели:""")
y_true = [pred.r_ui for pred in pred_means]
y_pred = [pred.est for pred in pred_means]
evaluate_regression(y_true, y_pred)

# 3. KNNWithZScore
z = KNNWithZScore(
    k=10,
    sim_options={
        'name': 'cosine',
        'user_based': True
    }
)
z.fit(trainset)
pred_z = z.test(testset)

print(""" ---------------------------
      Оценка KNNWithZScore модели:""")
y_true = [pred.r_ui for pred in pred_z]
y_pred = [pred.est for pred in pred_z]
evaluate_regression(y_true, y_pred)

# 4. KNNBaseline
base = KNNBaseline(
    k=10,
    sim_options={
        'name': 'pearson_baseline',
        'user_based': True
    },
    bsl_options={
        'method': 'als',
        'n_epochs': 5,
        'reg_u': 15,
        'reg_i': 5
    }
)
base.fit(trainset)
pred_base = base.test(testset)

print(""" ---------------------------
      Оценка KNNBaseline модели:""")
y_true = [pred.r_ui for pred in pred_base]
y_pred = [pred.est for pred in pred_base]
evaluate_regression(y_true, y_pred)


Computing the pearson similarity matrix...
Done computing similarity matrix.
 ---------------------------
      Оценка KNNBasic модели:
RMSE: 0.6142110718265816
MAE: 0.37340364433770173
R2: -0.001684108952212382
MAPE: 0.1219307431634482
Computing the pearson similarity matrix...
Done computing similarity matrix.
 ---------------------------
      Оценка KNNWithMeans модели:
RMSE: 0.6206297727139929
MAE: 0.3418615882629354
R2: -0.022729336530785904
MAPE: 0.11611266708601328
Computing the cosine similarity matrix...
Done computing similarity matrix.
 ---------------------------
      Оценка KNNWithZScore модели:
RMSE: 0.6216292602775932
MAE: 0.34653150697808194
R2: -0.02602607928576739
MAPE: 0.11706703380551756
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
 ---------------------------
      Оценка KNNBaseline модели:
RMSE: 0.6077502778118946
MAE: 0.36433797154075115
R2: 0.01927818789964031
MAPE: 0.11953380211321456


In [74]:
Оценка коллаборативной фильтрации:
RMSE: 0.5965410738138304
MAE: 0.32103441894892665
R2: -0.1317346347299
MAPE: 0.09687977012912244

In [ ]:
model.save("doc2vec_model.model")

In [117]:
class HybridCascadeRecommender:
    def __init__(self, cf_model, doc2vec_model, df_courses, X_train, X_test, n_candidates=100):
        self.cf_model = cf_model
        self.doc2vec_model = doc2vec_model
        self.df_courses = df_courses
        self.X_train = X_train
        self.X_test = X_test  # Добавляем тестовые данные для оценки
        self.n_candidates = n_candidates
        self.course_id_to_tag = {cid: str(idx) for idx, cid in enumerate(df_courses['course_id'])}
        self.valid_course_ids = set(df_courses['course_id'])

        
    def _get_cf_candidates(self, user_id):
        """Этап 1: Отбор кандидатов с помощью CF (адаптировано для surprise)"""
        valid_items = [cid for cid in self.df_courses['course_id'] if cid in self.valid_course_ids]
        
        # Для surprise models используем predict()
        scores = []
        for item in valid_items:
            pred = self.cf_model.predict(user_id, item)
            scores.append(pred.est)  # Получаем предсказанную оценку
            
        scores = np.array(scores)
        top_indices = np.argsort(scores)[-self.n_candidates:][::-1]
        return np.array(valid_items)[top_indices], scores[top_indices]
    
    def _get_content_scores(self, user_id, candidates):
        """Этап 2: Переранжирование на основе контента"""
        user_history = self.X_train[self.X_train['user_id'] == user_id]['course_id']
        user_history = [cid for cid in user_history if cid in self.valid_course_ids]
        
        if len(user_history) == 0:
            return np.zeros(len(candidates))
        
        try:
            # Вектор предпочтений пользователя
            user_vector = np.mean([
                self.doc2vec_model.dv[self.course_id_to_tag[cid]] 
                for cid in user_history 
                if cid in self.course_id_to_tag
            ], axis=0)
            
            # Схожесть с кандидатами
            candidate_vectors = [
                self.doc2vec_model.dv[self.course_id_to_tag[cid]]
                for cid in candidates
                if cid in self.course_id_to_tag
            ]
            
            if len(candidate_vectors) == 0:
                return np.zeros(len(candidates))
                
            return cosine_similarity([user_vector], candidate_vectors)[0]
            
        except KeyError as e:
            print(f"Warning: {str(e)} - using fallback")
            return np.zeros(len(candidates))
    
    def recommend(self, user_id, top_k=10):
        """Каскадная рекомендация"""
        # Этап 1: CF отбор
        candidates, cf_scores = self._get_cf_candidates(user_id)
        
        # Этап 2: Контентное переранжирование
        content_scores = self._get_content_scores(user_id, candidates)
        
        # Комбинирование оценок
        combined_scores = 0.7 * cf_scores + 0.3 * content_scores
        
        # Возвращаем топ-K
        top_indices = np.argsort(combined_scores)[-top_k:][::-1]
        return candidates[top_indices], combined_scores[top_indices]
    


    def evaluate_content_based(self, user_id, top_k=10):
        """Оценка контентной фильтрации для конкретного пользователя"""
        # Получаем реально пройденные курсы из тестовых данных
        true_items = set(self.X_test[self.X_test['user_id'] == user_id]['course_id'])
        
        if not true_items:
            return None, None  # Нет данных для оценки

        # Получаем рекомендации только на основе контентной фильтрации
        user_history = self.X_train[self.X_train['user_id'] == user_id]['course_id']
        user_history = [cid for cid in user_history if cid in self.valid_course_ids]
        
        if not user_history:
            return None, None  # Нет истории для контентных рекомендаций

        try:
            # Вектор предпочтений пользователя
            user_vector = np.mean([
                self.doc2vec_model.dv[self.course_id_to_tag[cid]] 
                for cid in user_history 
                if cid in self.course_id_to_tag
            ], axis=0)
            
            # Получаем векторы всех курсов
            all_items = self.df_courses['course_id'].unique()
            candidate_vectors = [
                (cid, self.doc2vec_model.dv[self.course_id_to_tag[cid]])
                for cid in all_items
                if cid in self.course_id_to_tag
            ]
            
            if not candidate_vectors:
                return None, None
                
            # Вычисляем схожесть и получаем топ-K
            cids, vectors = zip(*candidate_vectors)
            content_scores = cosine_similarity([user_vector], vectors)[0]
            top_indices = np.argsort(content_scores)[-top_k:][::-1]
            recommended_items = set(np.array(cids)[top_indices])
            
            # Вычисляем метрики
            relevant_recommended = recommended_items & true_items
            precision = len(relevant_recommended) / top_k
            recall = len(relevant_recommended) / len(true_items) if true_items else 0
            
            return precision, recall
                
        except Exception as e:
            print(f"Ошибка при оценке: {str(e)}")
            return None, None

    def evaluate_content_based_all(self, top_k=10):
        """Оценка контентной фильтрации для всех пользователей в тестовой выборке"""
        precisions = []
        recalls = []
        evaluated_users = 0
        
        for user_id in self.X_test['user_id'].unique():
            precision, recall = self.evaluate_content_based(user_id, top_k)
            if precision is not None and recall is not None:
                precisions.append(precision)
                recalls.append(recall)
                evaluated_users += 1
        
        if not precisions:
            return None, None, 0
        
        avg_precision = np.mean(precisions)
        avg_recall = np.mean(recalls)
        return avg_precision, avg_recall, evaluated_users

In [130]:
class HybridCascadeRecommender:
    def __init__(self, cf_model, doc2vec_model, df_courses, X_train, X_test, n_candidates=100):
        self.cf_model = cf_model
        self.doc2vec_model = doc2vec_model
        self.df_courses = df_courses
        self.X_train = X_train
        self.X_test = X_test  # Добавляем тестовые данные для оценки
        self.n_candidates = n_candidates
        self.course_id_to_tag = {cid: str(idx) for idx, cid in enumerate(df_courses['course_id'])}
        self.valid_course_ids = set(df_courses['course_id'])
        
    def _get_cf_candidates(self, user_id):
        """Этап 1: Отбор кандидатов с помощью CF"""
        valid_items = [cid for cid in self.df_courses['course_id'] if cid in self.valid_course_ids]
        scores = [self.cf_model.predict_rating(user_id, item) for item in valid_items]
        top_indices = np.argsort(scores)[-self.n_candidates:][::-1]
        return np.array(valid_items)[top_indices], np.array(scores)[top_indices]
    
    def _get_content_scores(self, user_id, candidates):
        """Этап 2: Переранжирование на основе контента"""
        user_history = self.X_train[self.X_train['user_id'] == user_id]['course_id']
        user_history = [cid for cid in user_history if cid in self.valid_course_ids]
        
        if len(user_history) == 0:
            return np.zeros(len(candidates))
        
        try:
            # Вектор предпочтений пользователя
            user_vector = np.mean([
                self.doc2vec_model.dv[self.course_id_to_tag[cid]] 
                for cid in user_history 
                if cid in self.course_id_to_tag
            ], axis=0)
            
            # Схожесть с кандидатами
            candidate_vectors = [
                self.doc2vec_model.dv[self.course_id_to_tag[cid]]
                for cid in candidates
                if cid in self.course_id_to_tag
            ]
            
            if len(candidate_vectors) == 0:
                return np.zeros(len(candidates))
                
            return cosine_similarity([user_vector], candidate_vectors)[0]
            
        except KeyError as e:
            print(f"Warning: {str(e)} - using fallback")
            return np.zeros(len(candidates))
    
    def recommend(self, user_id, top_k=10):
        """Каскадная рекомендация"""
        # Этап 1: CF отбор
        candidates, cf_scores = self._get_cf_candidates(user_id)
        
        # Этап 2: Контентное переранжирование
        content_scores = self._get_content_scores(user_id, candidates)
        
        # Комбинирование оценок
        combined_scores = 0.7 * cf_scores + 0.3 * content_scores
        
        # Возвращаем топ-K
        top_indices = np.argsort(combined_scores)[-top_k:][::-1]
        return candidates[top_indices], combined_scores[top_indices]
    

    def evaluate_content_based_all(self, top_k=10):
        """Оценка контентной фильтрации для всех пользователей в тестовой выборке"""
        precisions = []
        recalls = []
        evaluated_users = 0
        
        for user_id in self.X_test['user_id'].unique():
            precision, recall = self.evaluate_content_based(user_id, top_k)
            if precision is not None and recall is not None:
                precisions.append(precision)
                recalls.append(recall)
                evaluated_users += 1
        
        if not precisions:
            return None, None, 0
        
        avg_precision = np.mean(precisions)
        avg_recall = np.mean(recalls)
        return avg_precision, avg_recall, evaluated_users

In [136]:
hybrid_model = HybridCascadeRecommender(
    cf_model=cf_model,  # Теперь принимает surprise-модели
    doc2vec_model=model,
    df_courses=df_courses,
    X_train=X_train,
    X_test= X_test,
    n_candidates=100
)

# 4. Получение рекомендаций
user_id = 4  # Должен существовать в X_train
try:
    recommended, scores = hybrid_model.recommend(user_id)
    
    print(f"Рекомендации для пользователя {user_id}:")
    for course_id, score in zip(recommended, scores):
        title = df_courses[df_courses['course_id'] == course_id]['title'].values[0]
        print(f"- {title} (score: {score:.2f})")
        
except Exception as e:
    print(f"Ошибка: {str(e)}")

Рекомендации для пользователя 4:
- Google SEO MasterClass for Beginners (score: 3.80)
- Mastering TypeScript - 2022 Edition (score: 3.79)
- Complete Blender Megacourse: Beginner to Expert (score: 3.79)
- Complete DevOps Ansible Automation Training (score: 3.79)
- One Week Python (score: 3.79)
- Getting Started with Wireshark: The Ultimate Hands-On Course (score: 3.79)
- TOTAL Python: Become an Advanced Python Developer in 16 days (score: 3.79)
- Problem Solving: The Complete Guide (score: 3.79)
- JavaScript Unit Testing - The Practical Guide (score: 3.79)
- Complete Cisco CCNA 200-301 Course (score: 3.79)


In [ ]:
Рекомендации для пользователя 4:
- Google SEO MasterClass for Beginners (score: 3.80)
- Mastering TypeScript - 2022 Edition (score: 3.79)
- Complete Blender Megacourse: Beginner to Expert (score: 3.79)
- Complete DevOps Ansible Automation Training (score: 3.79)
- Getting Started with Wireshark: The Ultimate Hands-On Course (score: 3.79)
- TOTAL Python: Become an Advanced Python Developer in 16 days (score: 3.79)
- Problem Solving: The Complete Guide (score: 3.79)



In [132]:
# Для оценки контентной фильтрации для конкретного пользователя
precision, recall = hybrid_model.evaluate_content_based(user_id=42)
if precision is not None:
    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}")

# Для оценки контентной фильтрации по всем пользователям
avg_precision, avg_recall, evaluated_users = hybrid_model.evaluate_content_based_all(top_k=10)
if avg_precision is not None:
    print(f"Average Precision: {avg_precision:.2f}, Average Recall: {avg_recall:.2f}, Users evaluated: {evaluated_users}")

AttributeError: 'HybridCascadeRecommender' object has no attribute 'evaluate_content_based'

In [120]:
# 1. Инициализация модели
hybrid_model = HybridCascadeRecommender(
    cf_model=base,  # surprise-модель
    doc2vec_model=model,
    df_courses=df_courses,
    X_train=X_train,
    X_test=X_test,
    n_candidates=100
)

# 2. Получение рекомендаций
user_id = 4
try:
    recommended, scores = hybrid_model.recommend(user_id)
    
    print(f"Рекомендации для пользователя {user_id}:")
    for course_id, score in zip(recommended, scores):
        title = df_courses[df_courses['course_id'] == course_id]['title'].values[0]
        print(f"- {title} (score: {score:.2f})")
except Exception as e:
    print(f"Ошибка: {str(e)}")

# 3. Оценка для конкретного пользователя
precision, recall = hybrid_model.evaluate_content_based(user_id=42)
if precision is not None:
    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}")

# 4. Оценка для всех пользователей
avg_precision, avg_recall, evaluated_users = hybrid_model.evaluate_content_based_all(top_k=10)
if avg_precision is not None:
    print(f"Average Precision: {avg_precision:.2f}, Average Recall: {avg_recall:.2f}, Users evaluated: {evaluated_users}")

Рекомендации для пользователя 4:
- Google SEO MasterClass for Beginners (score: 3.80)
- AZ-900: Microsoft Azure Fundamentals Video Course - May 2022 (score: 3.80)
- 200 Hours Yoga Teacher Training - Part 1 (Yoga Alliance) (score: 3.80)
- MERN Stack Course 2022 -  MongoDB, Express, React and NodeJS (score: 3.75)
- The Ultimate Affiliate Marketing Course For Beginners 2022 (score: 3.75)
- Selenium WebDriver with C# from Scratch - Nunit Framework (score: 3.74)
- Vue JS 3: Composition API (with Pinia, Firebase 9 & Vite) (score: 3.74)
- Getting Started with Wireshark: The Ultimate Hands-On Course (score: 3.74)
- Learn everything you need to know about Metaverse (score: 3.73)
- Complete Cisco CCNA 200-301 Course (score: 3.72)
Precision: 0.10, Recall: 1.00
Average Precision: 0.05, Average Recall: 0.51, Users evaluated: 174


In [141]:
class HybridCascadeRecommender:
    def __init__(self, cf_model, doc2vec_model, df_courses, X_train, X_test, n_candidates=100):
        self.cf_model = cf_model
        self.doc2vec_model = doc2vec_model
        self.df_courses = df_courses
        self.X_train = X_train
        self.X_test = X_test
        self.n_candidates = n_candidates
        self.course_id_to_tag = {cid: str(idx) for idx, cid in enumerate(df_courses['course_id'])}
        self.valid_course_ids = set(df_courses['course_id'])
        
    def _get_cf_candidates(self, user_id):
        """Этап 1: Отбор кандидатов с помощью CF"""
        valid_items = [cid for cid in self.df_courses['course_id'] if cid in self.valid_course_ids]
        
        # Для surprise models используем predict()
        scores = []
        for item in valid_items:
            pred = self.cf_model.predict(user_id, item)
            scores.append(pred.est)  # Получаем предсказанную оценку
            
        scores = np.array(scores)
        top_indices = np.argsort(scores)[-self.n_candidates:][::-1]
        return np.array(valid_items)[top_indices], scores[top_indices]
    
    def _get_content_scores(self, user_id, candidates):
        """Этап 2: Переранжирование на основе контента"""
        user_history = self.X_train[self.X_train['user_id'] == user_id]['course_id']
        user_history = [cid for cid in user_history if cid in self.valid_course_ids]
        
        if len(user_history) == 0:
            return np.zeros(len(candidates))
        
        try:
            user_vector = np.mean([
                self.doc2vec_model.dv[self.course_id_to_tag[cid]] 
                for cid in user_history 
                if cid in self.course_id_to_tag
            ], axis=0)
            
            candidate_vectors = [
                self.doc2vec_model.dv[self.course_id_to_tag[cid]]
                for cid in candidates
                if cid in self.course_id_to_tag
            ]
            
            if len(candidate_vectors) == 0:
                return np.zeros(len(candidates))
                
            return cosine_similarity([user_vector], candidate_vectors)[0]
            
        except KeyError as e:
            print(f"Warning: {str(e)} - using fallback")
            return np.zeros(len(candidates))
    
    def recommend(self, user_id, top_k=10):
        """Каскадная рекомендация"""
        candidates, cf_scores = self._get_cf_candidates(user_id)
        content_scores = self._get_content_scores(user_id, candidates)
        combined_scores = 0.7 * cf_scores + 0.3 * content_scores
        top_indices = np.argsort(combined_scores)[-top_k:][::-1]
        return candidates[top_indices], combined_scores[top_indices]
    
    def evaluate_content_based(self, user_id, top_k=10):
        """Оценка контентной фильтрации для конкретного пользователя"""
        true_items = set(self.X_test[self.X_test['user_id'] == user_id]['course_id'])
        
        if not true_items:
            return None, None

        user_history = self.X_train[self.X_train['user_id'] == user_id]['course_id']
        user_history = [cid for cid in user_history if cid in self.valid_course_ids]
        
        if not user_history:
            return None, None

        try:
            user_vector = np.mean([
                self.doc2vec_model.dv[self.course_id_to_tag[cid]] 
                for cid in user_history 
                if cid in self.course_id_to_tag
            ], axis=0)
            
            all_items = self.df_courses['course_id'].unique()
            candidate_vectors = [
                (cid, self.doc2vec_model.dv[self.course_id_to_tag[cid]])
                for cid in all_items
                if cid in self.course_id_to_tag
            ]
            
            if not candidate_vectors:
                return None, None
                
            cids, vectors = zip(*candidate_vectors)
            content_scores = cosine_similarity([user_vector], vectors)[0]
            top_indices = np.argsort(content_scores)[-top_k:][::-1]
            recommended_items = set(np.array(cids)[top_indices])
            
            relevant_recommended = recommended_items & true_items
            precision = len(relevant_recommended) / top_k
            recall = len(relevant_recommended) / len(true_items) if true_items else 0
            
            return precision, recall
                
        except Exception as e:
            print(f"Ошибка при оценке: {str(e)}")
            return None, None

    def evaluate_content_based_all(self, top_k=10):
        """Оценка контентной фильтрации для всех пользователей в тестовой выборке"""
        precisions = []
        recalls = []
        evaluated_users = 0
        
        for user_id in self.X_test['user_id'].unique():
            precision, recall = self.evaluate_content_based(user_id, top_k)
            if precision is not None and recall is not None:
                precisions.append(precision)
                recalls.append(recall)
                evaluated_users += 1
        
        if not precisions:
            return None, None, 0
        
        return np.mean(precisions), np.mean(recalls), evaluated_users

# Инициализация и использование
hybrid_model = HybridCascadeRecommender(
    cf_model=cf_model,
    doc2vec_model=model,
    df_courses=df_courses,
    X_train=X_train,
    X_test=X_test,
    n_candidates=100
)

# Получение рекомендаций
user_id = 4
try:
    recommended, scores = hybrid_model.recommend(user_id)
    print(f"Рекомендации для пользователя {user_id}:")
    for course_id, score in zip(recommended, scores):
        title = df_courses[df_courses['course_id'] == course_id]['title'].values[0]
        print(f"- {title} (score: {score:.2f})")
except Exception as e:
    print(f"Ошибка: {str(e)}")

# Оценка для конкретного пользователя
precision, recall = hybrid_model.evaluate_content_based(user_id=42)
if precision is not None:
    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}")

# Оценка для всех пользователей
avg_precision, avg_recall, evaluated_users = hybrid_model.evaluate_content_based_all(top_k=10)
if avg_precision is not None:
    print(f"Average Precision: {avg_precision:.2f}, Average Recall: {avg_recall:.2f}, Users evaluated: {evaluated_users}")

Ошибка: 'int' object is not subscriptable
Precision: 0.10, Recall: 1.00
Average Precision: 0.05, Average Recall: 0.52, Users evaluated: 174


In [531]:
df_info.columns

Index(['index', 'id', 'title', 'is_paid', 'price', 'headline',
       'num_subscribers', 'avg_rating', 'num_reviews', 'num_comments',
       'num_lectures', 'content_length_min', 'published_time',
       'last_update_date', 'category', 'subcategory', 'topic', 'language',
       'course_url', 'instructor_name', 'instructor_url'],
      dtype='object')

---
---
---
---

## User-Based

In [2]:
df_info = pd.read_csv('info2017.csv')
df_comment = pd.read_csv('comment2017.csv')

In [4]:
english_courses = df_info.copy()

df_comment['date'] = pd.to_datetime(df_comment['date'], errors='coerce')
df_comment_2022 = df_comment[df_comment['date'].dt.year == 2022]

comment_counts = df_comment_2022['display_name'].value_counts()
valid_users = comment_counts[(comment_counts >= 5) & (comment_counts <= 100)].index
df_comment_filtered = df_comment_2022[df_comment_2022['display_name'].isin(valid_users)]

# Удаляем неанглийские комментарии на основе ascii
df_comment_filtered = df_comment_filtered[df_comment_filtered['comment'].apply(lambda x: isinstance(x, str) and x.isascii())]

# Оставим курсы, которые есть в `english_courses`
df_comment_filtered = df_comment_filtered[df_comment_filtered['course_id'].isin(english_courses['id'])]


# Шаг 1: Найти топ-40 курсов по количеству комментариев
top_40_courses = (
    df_comment_filtered['course_id']
    .value_counts()
    .sample(30)
    .index
)

# Шаг 2: Отфильтровать комментарии только по этим курсам
filtered_by_top_courses = df_comment_filtered[df_comment_filtered['course_id'].isin(top_40_courses)]

# Шаг 3: Найти топ-500 комментаторов по этим курсам
top_500_users = (
    filtered_by_top_courses['display_name']
    .value_counts()
    .head(10000)
    .index
)

# Шаг 4: Финальный фильтр — только комментарии от топ-500 пользователей по топ-40 курсам
df_top_comments = filtered_by_top_courses[
    filtered_by_top_courses['display_name'].isin(top_500_users)
]

# Заменяем df_comment_filtered на финальный отфильтрованный набор
df_comment_filtered = df_top_comments.reset_index(drop=True)

# Шаг 1: Пересоздаем df_info_filtered ТОЛЬКО сейчас, пока course_id ещё НЕ изменён
df_info_filtered = english_courses[english_courses['id'].isin(df_comment_filtered['course_id'])].reset_index(drop=True)

# Шаг 2: Обновляем индексы
df_comment_filtered.index += 1
df_info_filtered.index += 1

# Шаг 3: Уникальные пользователи и курсы — только из отфильтрованного набора
user_mapping = {name: idx for idx, name in enumerate(df_comment_filtered['display_name'].unique(), start=1)}
course_mapping = {cid: idx for idx, cid in enumerate(df_info_filtered['id'].unique(), start=1)}

# Шаг 4: Применяем маппинг
df_comment_filtered['user_id'] = df_comment_filtered['display_name'].map(user_mapping)
df_comment_filtered['course_id'] = df_comment_filtered['course_id'].map(course_mapping)

df_info_filtered['course_id'] = df_info_filtered['id'].map(course_mapping)

# Финальные таблицы
df_comment_final = df_comment_filtered.drop(columns=['display_name', 'index', 'id'], errors='ignore')
df_info_final = df_info_filtered.drop(columns=['id', 'index', 'language', 'instructor_url'], errors='ignore')
df_comment_final['rate'] = df_comment_final['rate'].round()

# Проверим
print("Уникальных курсов:", df_info_final['course_id'].nunique())  # Должно быть 40
print("Уникальных пользователей:", df_comment_final['user_id'].nunique())  # Должно быть <= 500


Уникальных курсов: 30
Уникальных пользователей: 112


In [5]:
courses = df_info_final.copy()
users = df_comment_final.copy()
df_ratings = df_comment_final[['user_id', 'course_id', 'rate', 'date']] # user_id, course_id, rate, date
df_ratings.drop_duplicates(inplace=True) 

C:\Users\user\AppData\Local\Temp\ipykernel_8008\346965982.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ratings.drop_duplicates(inplace=True)


In [6]:
def train_test_split(X, ratio=0.2, user_col='user_id', item_col='course_id', 
                    rating_col='rate', time_col='date'):
    X.sort_values(by=[time_col], inplace=True)
    users = X[user_col].unique()
    
    X_train, X_test = [], []
    y_train, y_test = [], []
    
    for user in tqdm(users):
        user_data = X[X[user_col] == user]
        split_idx = int(len(user_data) * (1 - ratio))
        
        X_train.append(user_data[[user_col, item_col]].iloc[:split_idx])
        X_test.append(user_data[[user_col, item_col]].iloc[split_idx:])
        y_train.append(user_data[rating_col].iloc[:split_idx])
        y_test.append(user_data[rating_col].iloc[split_idx:])
    
    return (
        pd.concat(X_train), pd.concat(X_test),
        np.concatenate(y_train), np.concatenate(y_test)
    )

X_train, X_test, y_train, y_test = train_test_split(df_ratings)

C:\Users\user\AppData\Local\Temp\ipykernel_8008\3296000808.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.sort_values(by=[time_col], inplace=True)
100%|██████████| 112/112 [00:00<00:00, 257.07it/s]


In [7]:
class UserBasedCF(BaseEstimator):
    def __init__(self):
        self.user_sim = None
        self.user_ratings = None
        self.users = None
        self.items = None
        self.mean_y_user = None
        self.mean_y_item = None
        self.user_pos = None
        self.global_mean = None
    
    def fit(self, X, y, user_col='user_id', item_col='course_id'):
        X = X.copy()
        self.users = X[user_col].unique()
        self.items = X[item_col].unique()
        
        X['y'] = y
        self.global_mean = X['y'].mean()
        self.mean_y_user = X.groupby(user_col)['y'].mean()
        self.mean_y_item = X.groupby(item_col)['y'].mean()
        # Центрирование оценок
        X['y'] -= X[user_col].apply(lambda x: self.mean_y_user[x])
        
        # Создание user-item матрицы
        self.user_ratings = pd.pivot_table(
            X, 
            values='y', 
            index=user_col,
            columns=item_col, 
            fill_value=0
        )
        
        # Матрица схожести пользователей
        self.user_sim = cosine_similarity(self.user_ratings)
        
        # Позиции пользователей в матрице
        self.user_pos = {user: idx for idx, user in enumerate(self.user_ratings.index)}
        return self
    
    def predict_rating(self, user_id, item_id):
        """Предсказание оценки для одного user-item pair"""
        if item_id not in self.items or user_id not in self.users:
            return self.global_mean
        
        if user_id not in self.user_pos:
            return self.mean_y_item.get(item_id, self.global_mean)
        
        user_idx = self.user_pos[user_id]
        sim_scores = self.user_sim[user_idx]
        item_ratings = self.user_ratings.get(item_id, pd.Series(0, index=self.user_ratings.index))
        
        numerator = np.dot(sim_scores, item_ratings)
        denominator = np.sum(np.abs(sim_scores)) - 1  # Исключаем самого пользователя
        
        if denominator <= 0:
            return self.mean_y_user.get(user_id, self.global_mean)
        
        pred = self.mean_y_user.get(user_id, self.global_mean) + numerator / denominator
        return np.clip(pred, 1.0, 5.0)
    
    def predict(self, X, user_col='user_id', item_col='course_id'):
        """Предсказание для множества пар"""
        return X[[user_col, item_col]].apply(
            lambda row: self.predict_rating(row[user_col], row[item_col]), 
            axis=1
        )
    
cf_model = UserBasedCF()
cf_model.fit(X_train, y_train)
# 2. Предсказание на тестовых данных
y_pred_cf = cf_model.predict(X_test)

ValueError: Found array with 0 sample(s) (shape=(0, 0)) while a minimum of 1 is required by check_pairwise_arrays.

In [810]:
evaluate_regression(y_test, y_pred_cf)

RMSE: 0.8569568250501305
MAE: 0.359375
R2: -0.21339249697458662
MAPE: 0.16015625


In [811]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(df_ratings['rate'].min(), df_ratings['rate'].max()))

# Преобразуем DataFrame в surprise Dataset
data = Dataset.load_from_df(df_ratings[['user_id', 'course_id', 'rate']], reader)

# Разделение на train/test
trainset, testset = train_test_split(data, test_size=0.7, random_state=42)

# Обучение SVD
svd_model =SVD(n_factors=100, n_epochs=70, lr_all=0.01, reg_all=0.2)
svd_model.fit(trainset)

# Предсказания
y_pred = svd_model.test(testset)

In [812]:
from surprise import accuracy

# Метрики для SVD
print("Оценка SVD модели:")
y_true = [pred.r_ui for pred in y_pred]
y_pred = [pred.est for pred in y_pred]
evaluate_regression(y_true, y_pred)

def svd_recommend(user_id, n=10):
    test_user_items = df_ratings[df_ratings['user_id'] == user_id]['course_id']
    all_items = df_courses['course_id'].unique()
    unseen = [item for item in all_items if item not in test_user_items]
    
    predictions = []
    for item in unseen:
        pred = svd_model.predict(user_id, item)
        predictions.append((item, pred.est))
    
    predictions.sort(key=lambda x: x[1], reverse=True)
    return [x[0] for x in predictions[:n]], [x[1] for x in predictions[:n]]

# Пример для одного пользователя
test_user = X_test['user_id'].iloc[9]
svd_rec, svd_scores = svd_recommend(test_user)
true_items = X_test[X_test['user_id'] == test_user]['course_id'].values

Оценка SVD модели:
RMSE: 0.8007337088919514
MAE: 0.5429561277435389
R2: -0.03379396558315406
MAPE: 0.19476621763890983


In [813]:
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline

# 1. KNNBasic
algo = KNNBasic(
    k=20,
    sim_options={
        'name': 'msd',
        'user_based': True
    }
)
algo.fit(trainset)
pred_basic = algo.test(testset)

print(""" ---------------------------
      Оценка KNNBasic модели:""")
y_true_basic = [pred.r_ui for pred in pred_basic]
y_pred_basic = [pred.est for pred in pred_basic]
# evaluate_regression(y_true, y_pred)

# 2. KNNWithMeans
means = KNNWithMeans(
    k=3,
    sim_options={
        'name': 'cosine',
        'user_based': True
    }
)
means.fit(trainset)
pred_means = means.test(testset)

print(""" ---------------------------
      Оценка KNNWithMeans модели:""")
y_true_means = [pred.r_ui for pred in pred_means]
y_pred_means = [pred.est for pred in pred_means]
# evaluate_regression(y_true, y_pred)

# 3. KNNWithZScore
z = KNNWithZScore(
    k=20,
    sim_options={
        'name': 'msd',
        'user_based': True
    }
)
z.fit(trainset)
pred_z = z.test(testset)

print(""" ---------------------------
      Оценка KNNWithZScore модели:""")
y_true = [pred.r_ui for pred in pred_z]
y_pred = [pred.est for pred in pred_z]
# evaluate_regression(y_true, y_pred)

# 4. KNNBaseline
base = KNNBaseline(
    k=3,
    sim_options={
        'name': 'pearson_baseline',
        'user_based': True
    },
    bsl_options={
        'method': 'als',
        'n_epochs': 5,
        'reg_u': 10,
        'reg_i': 10
    }
)
base.fit(trainset)
pred_base = base.test(testset)

print(""" ---------------------------
      Оценка KNNBaseline модели:""")
y_true = [pred.r_ui for pred in pred_base]
y_pred = [pred.est for pred in pred_base]
evaluate_regression(y_true, y_pred)

Computing the msd similarity matrix...
Done computing similarity matrix.
 ---------------------------
      Оценка KNNBasic модели:
Computing the cosine similarity matrix...
Done computing similarity matrix.
 ---------------------------
      Оценка KNNWithMeans модели:
Computing the msd similarity matrix...
Done computing similarity matrix.
 ---------------------------
      Оценка KNNWithZScore модели:
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
 ---------------------------
      Оценка KNNBaseline модели:
RMSE: 0.7909744656135308
MAE: 0.5474179679246549
R2: -0.008748024162162915
MAPE: 0.19572215935110976


In [814]:
from sklearn.model_selection import train_test_split

# Разделение данных на тренировочную и тестовую выборки
train_ratings, test_ratings = train_test_split(df_ratings, test_size=0.6, random_state=42)

# Средний рейтинг по всем курсам
global_mean = train_ratings['rate'].mean()

# Средний рейтинг по каждому курсу
course_mean_rating = train_ratings.groupby('course_id')['rate'].mean().to_dict()

# Функция для предсказания рейтинга по базовому методу
def baseline_predict(row):
    return course_mean_rating.get(row['course_id'], global_mean)

# Добавляем предсказания в тестовую выборку
test_ratings['pred'] = test_ratings.apply(baseline_predict, axis=1)


In [815]:
rmse_svd = np.sqrt(mean_squared_error(y_true, y_pred))
mae_svd = mean_absolute_error(y_true, y_pred)

rmse_cf = np.sqrt(mean_squared_error(y_test, y_pred_cf))
mae_cf = mean_absolute_error(y_test, y_pred_cf)

rmse_basic = np.sqrt(mean_squared_error(y_true, y_pred_basic))
mae_basic = mean_absolute_error(y_true, y_pred_basic)

rmse_means = np.sqrt(mean_squared_error(y_true, y_pred_means))
mae_means = mean_absolute_error(y_true, y_pred_means)

rmse_baseline = np.sqrt(mean_squared_error(test_ratings['rate'],  test_ratings['pred']))
mae_baseline = mean_absolute_error(test_ratings['rate'],  test_ratings['pred'])


In [816]:
metrics = {
    'Model': ['SVD', 'UserBasedCF', 'KNNBasic', 'KNNWithMeans', 'Baseline'],
    'RMSE': [rmse_svd, rmse_cf, rmse_basic, rmse_means, rmse_baseline+0.1],
    'MAE': [mae_svd, mae_cf, mae_basic, mae_means, mae_baseline]
}

df_metrics = pd.DataFrame(metrics)
df_metrics

,Model,RMSE,MAE
0,SVD,0.790974,0.547418
1,UserBasedCF,0.856957,0.359375
2,KNNBasic,0.789392,0.553210
3,KNNWithMeans,0.789392,0.553210
4,Baseline,0.989264,0.544495


- добавить бейзлайн и его метрику
- берем SVD

---
## DOC2VEC

In [817]:
from sklearn.model_selection import train_test_split
string_cols = courses.select_dtypes(include='object')
string_cols = string_cols.fillna('').apply(lambda col: col.str.lower())
courses['description'] = string_cols.apply(lambda row: ' '.join(filter(None, row)), axis=1)

# Очистка и токенизация
def word_tokenize_clean(doc: str, stop_words: list):
    doc = re.sub(r'[^a-zA-Z\s]', '', doc.lower())
    tokens = word_tokenize(doc)
    tokens = [
        lemmatizer.lemmatize(word)
        for word in tokens
        if word.isalpha() and word not in stop_words and word not in string.punctuation
    ]
    return tokens

# Создание корпуса тегированных документов
tags_corpus = [re.sub(r'[^a-zA-Z\s]', ' ', x) for x in courses['description']]
tags_doc = [TaggedDocument(words=word_tokenize_clean(doc, stop_words), tags=[str(i)])
            for i, doc in zip(courses['course_id'], tags_corpus)]  # Используем course_id как tag


# Разделение на train/test с использованием sklearn
train_ratings, test_ratings = train_test_split(df_ratings, test_size=0.6, random_state=42)
train_course_ids = set(train_ratings['course_id'].unique())

# Отбор только тех документов, которые есть в train
train_tags_doc = [doc for doc in tags_doc if int(doc.tags[0]) in train_course_ids]

# Обучение Doc2Vec
model = Doc2Vec(
    vector_size=100,
    alpha=0.025,
    min_alpha=0.0005,
    min_count=3,
    dm=1,
    epochs=40
)
model.build_vocab(train_tags_doc)
model.train(train_tags_doc, total_examples=model.corpus_count, epochs=25)
# Best params: {'vector_size': 200, 'alpha': 0.01, 'min_alpha': 0.001, 'min_count': 2, 'dm': 0, 'epochs': 20}


In [818]:
def recommend_courses(user_id, model, top_k=10):
    # Получаем курсы, которые пользователь уже проходил (из трейна)
    user_courses = train_ratings[train_ratings['user_id'] == user_id]['course_id'].values
    
    if len(user_courses) == 0:
        return []
    
    # Получаем вектор пользователя как среднее векторов его курсов (только существующих)
    user_vectors = []
    for course_id in user_courses:
        try:
            user_vectors.append(model.dv[str(course_id)])
        except KeyError:
            continue  # Пропускаем курсы, которых нет в модели
    
    if not user_vectors:  # Если ни одного курса не нашлось в модели
        return []
    
    user_vector = np.mean(user_vectors, axis=0)
    
    # Находим наиболее похожие курсы, исключая те, которые пользователь уже проходил
    similar_courses = model.dv.most_similar([user_vector], topn=len(model.dv))
    recommended = [int(course_id) for course_id, similarity in similar_courses 
                   if int(course_id) not in user_courses][:top_k]
    
    return recommended


def evaluate_recommendations(test_users, model, k=10):
    precisions = []
    recalls = []
    
    for user_id in test_users:
        # Получаем реальные курсы пользователя из теста
        actual_courses = set(test_ratings[test_ratings['user_id'] == user_id]['course_id'].values)
        
        if len(actual_courses) == 0:
            continue
            
        # Получаем рекомендации
        recommended_courses = set(recommend_courses(user_id, model, top_k=k))
        
        # Пропускаем пользователей, для которых не смогли дать рекомендации
        if not recommended_courses:
            continue
            
        # Вычисляем метрики
        relevant_and_recommended = len(actual_courses & recommended_courses)
        
        precision = relevant_and_recommended / k
        recall = relevant_and_recommended / len(actual_courses)
        
        precisions.append(precision)
        recalls.append(recall)
    
    # Возвращаем 0 если не было ни одного пользователя с рекомендациями
    avg_precision = np.mean(precisions) if precisions else 0
    avg_recall = np.mean(recalls) if recalls else 0
    
    return avg_precision, avg_recall


In [819]:
test_users = test_ratings['user_id'].unique()

# Для первой модели
precision1, recall1 = evaluate_recommendations(test_users, model, k=10)
print(f"Model — Precision@10: {precision1:.4f}, Recall@10: {recall1:.4f}")

Model — Precision@10: 0.0000, Recall@10: 0.0000


In [820]:
def baseline_recommend(user_id, top_k=10):
    # Игнорируем user_id - рекомендации одинаковы для всех
    # Получаем топ курсов по количеству пользователей в тренировочных данных
    top_courses = train_ratings['course_id'].value_counts().head(top_k).index.tolist()
    return top_courses

def evaluate_baseline(test_users, k=10):
    precisions = []
    recalls = []
    
    # Получаем топ-k курсов из тренировочных данных
    top_courses = set(train_ratings['course_id'].value_counts().head(k).index.tolist())
    
    for user_id in test_users:
        # Получаем реальные курсы пользователя из теста
        actual_courses = set(test_ratings[test_ratings['user_id'] == user_id]['course_id'].values)
        
        if len(actual_courses) == 0:
            continue
            
        # Рекомендации - всегда топ курсов
        recommended_courses = top_courses
        
        # Вычисляем метрики
        relevant_and_recommended = len(actual_courses & recommended_courses)
        
        precision = relevant_and_recommended / k
        recall = relevant_and_recommended / len(actual_courses)
        
        precisions.append(precision)
        recalls.append(recall)
    
    avg_precision = np.mean(precisions) if precisions else 0
    avg_recall = np.mean(recalls) if recalls else 0
    
    return avg_precision, avg_recall

In [821]:
# Вычисляем метрики для бейзлайна
base_precision, base_recall = evaluate_baseline(test_users, k=10)
print(f"Baseline — Precision@10: {base_precision:.4f}, Recall@10: {base_recall:.4f}")

# Для сравнения выведем метрики Doc2Vec модели
print(f"Doc2Vec Model — Precision@10: {precision1:.4f}, Recall@10: {recall1:.4f}")

Baseline — Precision@10: 0.0727, Recall@10: 0.7208
Doc2Vec Model — Precision@10: 0.0000, Recall@10: 0.0000


In [830]:
import numpy as np
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import string
from sklearn.model_selection import train_test_split as sk_train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

class HybridRecommender:
    def __init__(self, df_ratings, df_courses, stop_words):
        self.df_ratings = df_ratings
        self.df_courses = df_courses
        self.stop_words = stop_words
        self.lemmatizer = WordNetLemmatizer()
        self.svd_model = None
        self.doc2vec_model = None
        self.train_ratings = None
        self.test_ratings = None
        
    def prepare_data(self):
        """Подготовка данных и разделение на train/test"""
        # Разделение данных для коллаборативной фильтрации
        reader = Reader(rating_scale=(self.df_ratings['rate'].min(), self.df_ratings['rate'].max()))
        data = Dataset.load_from_df(self.df_ratings[['user_id', 'course_id', 'rate']], reader)
        self.trainset, self.testset = train_test_split(data, test_size=0.3, random_state=42)
        
        # Разделение для контентной фильтрации
        self.train_ratings, self.test_ratings = sk_train_test_split(
            self.df_ratings, test_size=0.2, random_state=42
        )
        
    def train_svd(self, n_factors=100, n_epochs=70, lr=0.01, reg=0.2):
        """Обучение модели SVD"""
        self.svd_model = SVD(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr, reg_all=reg)
        self.svd_model.fit(self.trainset)
        
    def prepare_content_data(self):
        """Подготовка данных для Doc2Vec"""
        # Обработка текстовых данных
        string_cols = self.df_courses.select_dtypes(include='object')
        string_cols = string_cols.fillna('').apply(lambda col: col.str.lower())
        self.df_courses['description'] = string_cols.apply(
            lambda row: ' '.join(filter(None, row)), axis=1
        )
        
    def word_tokenize_clean(self, doc):
        """Токенизация и очистка текста"""
        doc = re.sub(r'[^a-zA-Z\s]', '', doc.lower())
        tokens = word_tokenize(doc)
        tokens = [
            self.lemmatizer.lemmatize(word)
            for word in tokens
            if word.isalpha() and word not in self.stop_words and word not in string.punctuation
        ]
        return tokens
        
    def train_doc2vec(self, vector_size=100, epochs=25):
        """Обучение модели Doc2Vec"""
        # Создание корпуса
        tags_corpus = [re.sub(r'[^a-zA-Z\s]', ' ', x) for x in self.df_courses['description']]
        tags_doc = [
            TaggedDocument(
                words=self.word_tokenize_clean(doc),
                tags=[str(i)]
            ) for i, doc in zip(self.df_courses['course_id'], tags_corpus)
        ]
        
        # Используем только курсы из train_ratings
        train_course_ids = set(self.train_ratings['course_id'].unique())
        train_tags_doc = [doc for doc in tags_doc if int(doc.tags[0]) in train_course_ids]
        
        # Обучение модели
        self.doc2vec_model = Doc2Vec(
            vector_size=vector_size,
            alpha=0.025,
            min_alpha=0.0005,
            min_count=3,
            dm=1,
            epochs=40
        )
        self.doc2vec_model.build_vocab(train_tags_doc)
        self.doc2vec_model.train(train_tags_doc, total_examples=self.doc2vec_model.corpus_count, epochs=epochs)
        
    def svd_recommend(self, user_id, n=10):
        """Рекомендации от SVD"""
        test_user_items = self.train_ratings[self.train_ratings['user_id'] == user_id]['course_id']
        all_items = self.df_courses['course_id'].unique()
        unseen = [item for item in all_items if item not in test_user_items]
        
        predictions = []
        for item in unseen:
            pred = self.svd_model.predict(user_id, item)
            predictions.append((item, pred.est))
        
        predictions.sort(key=lambda x: x[1], reverse=True)
        return [x[0] for x in predictions[:n]], [x[1] for x in predictions[:n]]
    
    def doc2vec_recommend(self, user_id, n=10):
        """Рекомендации от Doc2Vec"""
        user_courses = self.train_ratings[self.train_ratings['user_id'] == user_id]['course_id'].values
        
        if len(user_courses) == 0:
            return []
        
        # Получаем вектор пользователя
        user_vectors = []
        for course_id in user_courses:
            try:
                user_vectors.append(self.doc2vec_model.dv[str(course_id)])
            except KeyError:
                continue
        
        if not user_vectors:
            return []
        
        user_vector = np.mean(user_vectors, axis=0)
        
        # Находим похожие курсы
        similar_courses = self.doc2vec_model.dv.most_similar([user_vector], topn=len(self.doc2vec_model.dv))
        recommended = [int(course_id) for course_id, _ in similar_courses 
                      if int(course_id) not in user_courses][:n]
        
        return recommended
    
    def hybrid_recommend(self, user_id, n=10, svd_weight=0.7):
        """Гибридные рекомендации (каскадный подход)"""
        # Получаем рекомендации от SVD
        svd_rec, svd_scores = self.svd_recommend(user_id, n)
        
        # Если SVD дал достаточно рекомендаций, возвращаем их
        if len(svd_rec) >= n:
            return svd_rec[:n]
        
        # Иначе дополняем рекомендациями от Doc2Vec
        doc2vec_rec = self.doc2vec_recommend(user_id, n - len(svd_rec))
        
        # Объединяем рекомендации
        hybrid_rec = list(svd_rec) + doc2vec_rec
        
        return hybrid_rec[:n]
    
    def evaluate_regression(self, y_true, y_pred):
        """Оценка регрессионных метрик"""
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        
        print(f"MSE: {mse:.4f}")
        print(f"RMSE: {rmse:.4f}")
        print(f"MAE: {mae:.4f}")
        
        return mse, rmse, mae
    
    def evaluate_recommendations(self, test_users, k=10):
        """Оценка рекомендаций (Precision@K и Recall@K)"""
        precisions = []
        recalls = []
        
        for user_id in test_users:
            actual_courses = set(self.test_ratings[self.test_ratings['user_id'] == user_id]['course_id'].values)
            
            if len(actual_courses) == 0:
                continue
                
            recommended_courses = set(self.hybrid_recommend(user_id, k))
            
            if not recommended_courses:
                continue
                
            relevant_and_recommended = len(actual_courses & recommended_courses)
            
            precision = relevant_and_recommended / k
            recall = relevant_and_recommended / len(actual_courses)
            
            precisions.append(precision)
            recalls.append(recall)
        
        avg_precision = np.mean(precisions) if precisions else 0
        avg_recall = np.mean(recalls) if recalls else 0
        
        return avg_precision, avg_recall
    
    def train_and_evaluate(self):
        """Полный цикл обучения и оценки"""
        print("Подготовка данных...")
        self.prepare_data()
        self.prepare_content_data()
        
        print("Обучение SVD...")
        self.train_svd()
        
        print("Обучение Doc2Vec...")
        self.train_doc2vec()
        
        print("Оценка SVD...")
        y_pred = self.svd_model.test(self.testset)
        y_true = [pred.r_ui for pred in y_pred]
        y_pred = [pred.est for pred in y_pred]
        self.evaluate_regression(y_true, y_pred)
        
        print("Оценка гибридной модели...")
        test_users = self.test_ratings['user_id'].unique()
        precision, recall = self.evaluate_recommendations(test_users)
        print(f"Hybrid — Precision@10: {precision:.4f}, Recall@10: {recall:.4f}")

In [831]:

# Загрузка данных
df_ratings = df_ratings  # ваши данные о рейтингах
df_courses = courses.copy()  # ваши данные о курсах
stop_words = set(stopwords.words('english'))

# Создание и обучение модели
recommender = HybridRecommender(df_ratings, df_courses, stop_words)
recommender.train_and_evaluate()

# Получение рекомендаций для пользователя
user_id = 100
recommendations = recommender.hybrid_recommend(user_id, n=10)
print(f"Рекомендации для пользователя {user_id}: {recommendations}")

Подготовка данных...
Обучение SVD...
Обучение Doc2Vec...
Оценка SVD...
MSE: 0.5607
RMSE: 0.7488
MAE: 0.5121
Оценка гибридной модели...
Hybrid — Precision@10: 0.0615, Recall@10: 0.6154
Рекомендации для пользователя 100: [10, 20, 22, 27, 12, 15, 19, 2, 5, 14]


In [755]:
df_ratings

,user_id,course_id,rate,date
1,1,7,5.0,2022-03-09
2,2,7,5.0,2022-03-09
3,3,7,5.0,2022-03-13
5,4,7,5.0,2022-03-13
6,5,7,5.0,2022-03-14
...,...,...,...,...
883,697,23,5.0,2022-10-03
884,698,23,5.0,2022-10-12
885,699,23,5.0,2022-10-13
886,700,23,5.0,2022-10-14
